In [1]:
!pip install "numpy<2.3.0" --prefer-binary

In [2]:
!pip install pi-heif --only-binary=:all:

In [3]:
!pip install rfdetr  

In [4]:
import rfdetr
from rfdetr import RFDETRXLarge

import boto3
import os

In [5]:
sts = boto3.client("sts")
sts.get_caller_identity()

{'UserId': 'AROAYUUD7Z2HCA4ETIYGZ:SageMaker',
 'Account': '594056040078',
 'Arn': 'arn:aws:sts::594056040078:assumed-role/sotaaie2einnovation/SageMaker',
 'ResponseMetadata': {'RequestId': 'e3cddad2-16c4-4ff6-b68d-de1323ed3290',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e3cddad2-16c4-4ff6-b68d-de1323ed3290',
   'x-amz-sts-extended-request-id': 'MTp1cy1lYXN0LTE6UzoxNzY5OTIxNDY0MTQ4OlI6bmJxeTV1OU4=',
   'content-type': 'text/xml',
   'content-length': '444',
   'date': 'Sun, 01 Feb 2026 04:51:04 GMT'},
  'RetryAttempts': 0}}

In [6]:
import sagemaker
sagemaker.get_execution_role()

Unable to load JumpStart region config.
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/jumpstart/constants.py", line 69, in _load_region_config
    with open(filepath) as f:
FileNotFoundError: [Errno 2] No such file or directory: '/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/jumpstart/region_config.json'


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


'arn:aws:iam::594056040078:role/sotaaie2einnovation'

In [7]:
# Download coco.zip from S3 RF DETR Sentinel 3 bucket to local file directory

s3_uri = "s3://rfdetr-s3-sota-to-sam3-agent-docs-gmail/xView Tiled.v2i.coco.zip"
bucket = "rfdetr-s3-sota-to-sam3-agent-docs-gmail"
key = "xView Tiled.v2i.coco.zip"
zip_local = "xView_Object_Det_RFDETR.zip"

s3 = boto3.client('s3')
s3.download_file(bucket, key, zip_local)

In [8]:
s3.head_object(
    Bucket="rfdetr-s3-sota-to-sam3-agent-docs-gmail",
    Key="xView Tiled.v2i.coco.zip"
)


{'ResponseMetadata': {'RequestId': 'AA0BJYRDKWJKRHAB',
  'HostId': '1FTVLcxoUHVoB3hyrmFPHSndde/yH0CX94qVrwx8GW6/teiUjhFtxB4AHQKgiZDALKbadHW+l/Q=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '1FTVLcxoUHVoB3hyrmFPHSndde/yH0CX94qVrwx8GW6/teiUjhFtxB4AHQKgiZDALKbadHW+l/Q=',
   'x-amz-request-id': 'AA0BJYRDKWJKRHAB',
   'date': 'Sun, 01 Feb 2026 04:51:19 GMT',
   'last-modified': 'Sat, 31 Jan 2026 15:50:31 GMT',
   'etag': '"a80c1415986aa37c525d55e70c4a65c4-64"',
   'x-amz-server-side-encryption': 'AES256',
   'accept-ranges': 'bytes',
   'content-type': 'application/zip',
   'content-length': '1090885581',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2026, 1, 31, 15, 50, 31, tzinfo=tzutc()),
 'ContentLength': 1090885581,
 'ETag': '"a80c1415986aa37c525d55e70c4a65c4-64"',
 'ContentType': 'application/zip',
 'ServerSideEncryption': 'AES256',
 'Metadata': {}}

In [9]:
!pwd
!ls -lh xView_Object_Det_RFDETR.zip


/home/ec2-user/SageMaker
-rw-rw-r-- 1 ec2-user ec2-user 1.1G Feb  1 04:51 xView_Object_Det_RFDETR.zip


In [10]:
# !unzip xView_Object_Det_RFDETR.zip -d xview_coco

In [11]:
# Initialize RF-DETR Extra Large
model = RFDETRXLarge(accept_platform_model_license=True)

Using a different number of positional encodings than DINOv2, which means we're not loading DINOv2 backbone weights. This is not a problem if finetuning a pretrained RF-DETR model.
Using patch size 20 instead of 14, which means we're not loading DINOv2 backbone weights. This is not a problem if finetuning a pretrained RF-DETR model.
Loading pretrain weights


Reinitializing detection head with 91 classes


In [ ]:
model.train(
    dataset_dir="xview_coco",
    epochs=10,
    early_stopping=True,
    patience=3,
    min_delta=0.01,
    batch_size=1,
    grad_accum_steps=2,    # Total effective batch still 16
    lr=1e-4, # Recommended Learning Rate
    weight_decay=1e-10, # Recommended Weight Decay
    resolution=560,
    use_ema=False, # Expoentnial Moving Weights
    output_dir="RFDETRxLargeXView"
)
    

Reinitializing your detection head with 61 classes.
fatal: not a git repository (or any of the parent directories): .git


Unable to initialize TensorBoard. Logging is turned off for this session.  Run 'pip install tensorboard' to enable logging.
Not using distributed mode
git:
  sha: N/A, status: clean, branch: N/A

Namespace(num_classes=61, grad_accum_steps=2, print_freq=10, amp=True, lr=0.0001, lr_encoder=0.00015, batch_size=1, weight_decay=1e-10, epochs=10, lr_drop=100, clip_max_norm=0.1, lr_vit_layer_decay=0.8, lr_component_decay=0.7, do_benchmark=False, dropout=0, drop_path=0.0, drop_mode='standard', drop_schedule='constant', cutoff_epoch=0, pretrained_encoder=None, pretrain_weights='rf-detr-xlarge.pth', pretrain_exclude_keys=None, pretrain_keys_modify_to_load=None, pretrained_distiller=None, encoder='dinov2_windowed_base', vit_encoder_num_layers=12, window_block_indexes=None, position_embedding='sine', out_feature_indexes=[3, 6, 9, 12], freeze_encoder=False, layer_norm=True, rms_norm=False, backbone_lora=False, force_no_pretrain=False, dec_layers=5, dim_feedforward=2048, hidden_dim=512, sa_nheads=16

Epoch: [0]  [   0/8274]  eta: 3:30:28  lr: 0.000100  class_error: 99.59  loss: 15.0837 (15.0837)  loss_ce: 0.4740 (0.4740)  loss_bbox: 0.3323 (0.3323)  loss_giou: 1.6760 (1.6760)  loss_ce_0: 0.4394 (0.4394)  loss_bbox_0: 0.3749 (0.3749)  loss_giou_0: 1.7287 (1.7287)  loss_ce_1: 0.4435 (0.4435)  loss_bbox_1: 0.3627 (0.3627)  loss_giou_1: 1.7045 (1.7045)  loss_ce_2: 0.4475 (0.4475)  loss_bbox_2: 0.3458 (0.3458)  loss_giou_2: 1.6992 (1.6992)  loss_ce_3: 0.4688 (0.4688)  loss_bbox_3: 0.3324 (0.3324)  loss_giou_3: 1.6771 (1.6771)  loss_ce_enc: 0.4282 (0.4282)  loss_bbox_enc: 0.3860 (0.3860)  loss_giou_enc: 1.7627 (1.7627)  loss_ce_unscaled: 0.4740 (0.4740)  class_error_unscaled: 99.5870 (99.5870)  loss_bbox_unscaled: 0.0665 (0.0665)  loss_giou_unscaled: 0.8380 (0.8380)  cardinality_error_unscaled: 3751.0000 (3751.0000)  loss_ce_0_unscaled: 0.4394 (0.4394)  loss_bbox_0_unscaled: 0.0750 (0.0750)  loss_giou_0_unscaled: 0.8643 (0.8643)  cardinality_error_0_unscaled: 3751.0000 (3751.0000)  loss_

## **Training Logs: Achieved ~15.2 mAP@50 and ~7.6 mAP@[.5:.95] after early-stage training**

In [ ]:
{"train_lr": 9.999999999999097e-05, "train_class_error": 27.641346090278507, "train_loss": 12.117268059107488, "train_loss_ce": 0.7986306571467222, "train_loss_bbox": 0.21292840312771968, "train_loss_giou": 0.988178468025016, "train_loss_ce_0": 0.7936606122197591, "train_loss_bbox_0": 0.22226127472647383, "train_loss_giou_0": 1.016647588767801, "train_loss_ce_1": 0.7921367900305977, "train_loss_bbox_1": 0.2186026979118031, "train_loss_giou_1": 1.0071217933611605, "train_loss_ce_2": 0.7932020971629598, "train_loss_bbox_2": 0.2159259534091548, "train_loss_giou_2": 1.0012852082018628, "train_loss_ce_3": 0.7963058206945336, "train_loss_bbox_3": 0.21404413705991557, "train_loss_giou_3": 0.993980227777109, "train_loss_ce_enc": 0.7774885031536486, "train_loss_bbox_enc": 0.23084515353650348, "train_loss_giou_enc": 1.0440226635236742, "train_loss_ce_unscaled": 0.7986306571467222, "train_class_error_unscaled": 27.641346090278507, "train_loss_bbox_unscaled": 0.04258568065321685, "train_loss_giou_unscaled": 0.494089234012508, "train_cardinality_error_unscaled": 3831.560913705584, "train_loss_ce_0_unscaled": 0.7936606122197591, "train_loss_bbox_0_unscaled": 0.04445225497514571, "train_loss_giou_0_unscaled": 0.5083237943839005, "train_cardinality_error_0_unscaled": 3831.9158810732415, "train_loss_ce_1_unscaled": 0.7921367900305977, "train_loss_bbox_1_unscaled": 0.04372053961412509, "train_loss_giou_1_unscaled": 0.5035608966805802, "train_cardinality_error_1_unscaled": 3832.2277012327772, "train_loss_ce_2_unscaled": 0.7932020971629598, "train_loss_bbox_2_unscaled": 0.04318519070659419, "train_loss_giou_2_unscaled": 0.5006426041009314, "train_cardinality_error_2_unscaled": 3831.9039158810733, "train_loss_ce_3_unscaled": 0.7963058206945336, "train_loss_bbox_3_unscaled": 0.0428088274179094, "train_loss_giou_3_unscaled": 0.4969901138885545, "train_cardinality_error_3_unscaled": 3831.836354846507, "train_loss_ce_enc_unscaled": 0.7774885031536486, "train_loss_bbox_enc_unscaled": 0.04616903073312199, "train_loss_giou_enc_unscaled": 0.5220113317618371, "train_cardinality_error_enc_unscaled": 3831.9031907179115, "test_class_error": 21.001781582302517, "test_loss": 11.00145106451852, "test_loss_ce": 0.8157406374007936, "test_loss_bbox": 0.15601556807697292, "test_loss_giou": 0.8397759849211527, "test_loss_ce_0": 0.8196750992063492, "test_loss_bbox_0": 0.16470003822847964, "test_loss_giou_0": 0.867288599846855, "test_loss_ce_1": 0.807644779265873, "test_loss_bbox_1": 0.16173675447584146, "test_loss_giou_1": 0.8595148283905453, "test_loss_ce_2": 0.8118799603174603, "test_loss_bbox_2": 0.15845823953844726, "test_loss_giou_2": 0.8509355400857471, "test_loss_ce_3": 0.814949466765873, "test_loss_bbox_3": 0.1577457461600739, "test_loss_giou_3": 0.8433632152610355, "test_loss_ce_enc": 0.793679935515873, "test_loss_bbox_enc": 0.17515061603297316, "test_loss_giou_enc": 0.9031960644892284, "test_loss_ce_unscaled": 0.8157406374007936, "test_class_error_unscaled": 21.001781582302517, "test_loss_bbox_unscaled": 0.031203113665124255, "test_loss_giou_unscaled": 0.41988799246057634, "test_cardinality_error_unscaled": 236.7904761904762, "test_loss_ce_0_unscaled": 0.8196750992063492, "test_loss_bbox_0_unscaled": 0.03294000763327829, "test_loss_giou_0_unscaled": 0.4336442999234275, "test_cardinality_error_0_unscaled": 236.8031746031746, "test_loss_ce_1_unscaled": 0.807644779265873, "test_loss_bbox_1_unscaled": 0.03234735094749975, "test_loss_giou_1_unscaled": 0.42975741419527264, "test_cardinality_error_1_unscaled": 236.80190476190475, "test_loss_ce_2_unscaled": 0.8118799603174603, "test_loss_bbox_2_unscaled": 0.03169164790964079, "test_loss_giou_2_unscaled": 0.42546777004287356, "test_cardinality_error_2_unscaled": 236.80190476190475, "test_loss_ce_3_unscaled": 0.814949466765873, "test_loss_bbox_3_unscaled": 0.03154914921977454, "test_loss_giou_3_unscaled": 0.42168160763051776, "test_cardinality_error_3_unscaled": 236.7993650793651, "test_loss_ce_enc_unscaled": 0.793679935515873, "test_loss_bbox_enc_unscaled": 0.03503012318134544, "test_loss_giou_enc_unscaled": 0.4515980322446142, "test_cardinality_error_enc_unscaled": 236.79619047619047, "test_results_json": {"class_map": [{"class": "Aircraft Hangar", "map@50:95": 0.005002073962103408, "map@50": 0.007879746968382414, "precision": 0.01904761904761905, "recall": 0.08333333333333333, "f1_score": 0.031007751937984496}, {"class": "Barge", "map@50:95": 0.02396646663565853, "map@50": 0.036176814698779716, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Building", "map@50:95": 0.23044377716440137, "map@50": 0.5427051491202084, "precision": 0.3753249668019526, "recall": 0.6790457660096438, "f1_score": 0.4834406374406014}, {"class": "Bus", "map@50:95": 0.05914635510136962, "map@50": 0.15548753462122386, "precision": 0.2349624060150376, "recall": 0.35335689045936397, "f1_score": 0.28224668360146776}, {"class": "Cargo Car", "map@50:95": 0.2535609073130326, "map@50": 0.5467831545263404, "precision": 0.282798833819242, "recall": 0.7348484848484849, "f1_score": 0.4084210526315789}, {"class": "Cargo Plane", "map@50:95": 0.49716761461048764, "map@50": 0.8355498556673799, "precision": 0.5231788079470199, "recall": 0.8876404494382022, "f1_score": 0.6583333333333333}, {"class": "Cargo Truck", "map@50:95": 0.00959903778012376, "map@50": 0.02956903275779357, "precision": 0.08600528102602792, "recall": 0.19672131147540983, "f1_score": 0.11968503937007877}, {"class": "Cement Mixer", "map@50:95": 7.388798581350672e-05, "map@50": 0.00014777597162701344, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Construction Site", "map@50:95": 0.0026216849603589256, "map@50": 0.006707263296193631, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Container Crane", "map@50:95": 0.07374085234610415, "map@50": 0.1278519156263452, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Container Ship", "map@50:95": 0.2683790219877632, "map@50": 0.3592349559411367, "precision": 0.5263157894736842, "recall": 0.38461538461538464, "f1_score": 0.4444444444444444}, {"class": "Crane Truck", "map@50:95": 0.000988393412984709, "map@50": 0.0020723002532811417, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Damaged Building", "map@50:95": 0.024689667658971997, "map@50": 0.052625838071247585, "precision": 0.2169811320754717, "recall": 0.08243727598566308, "f1_score": 0.11948051948051949}, {"class": "Dump Truck", "map@50:95": 0.044760605844531065, "map@50": 0.10503605302183845, "precision": 0.15265486725663716, "recall": 0.23154362416107382, "f1_score": 0.18399999999999997}, {"class": "Engineering Vehicle", "map@50:95": 0.020195974703289595, "map@50": 0.05850221025871183, "precision": 1.0, "recall": 0.022222222222222223, "f1_score": 0.04347826086956522}, {"class": "Excavator", "map@50:95": 0.10343386955395364, "map@50": 0.24690590172652094, "precision": 0.12345679012345678, "recall": 0.4945054945054945, "f1_score": 0.19758507135016465}, {"class": "Facility", "map@50:95": 0.0803282467604919, "map@50": 0.13738749311715712, "precision": 0.1590909090909091, "recall": 0.3211009174311927, "f1_score": 0.2127659574468085}, {"class": "Ferry", "map@50:95": 0.05007206624930869, "map@50": 0.0671468955575845, "precision": 0.06060606060606061, "recall": 0.25, "f1_score": 0.0975609756097561}, {"class": "Fishing Vessel", "map@50:95": 0.06845985276664567, "map@50": 0.12905807201088976, "precision": 0.18823529411764706, "recall": 0.2077922077922078, "f1_score": 0.19753086419753088}, {"class": "Fixed-wing Aircraft", "map@50:95": 0.003465346534653466, "map@50": 0.0049504950495049506, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Flat Car", "map@50:95": 0.0, "map@50": 0.0, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Front loader-Bulldozer", "map@50:95": 0.03458381141608866, "map@50": 0.08324307112267887, "precision": 0.102880658436214, "recall": 0.17857142857142858, "f1_score": 0.13054830287206265}, {"class": "Ground Grader", "map@50:95": 0.008575577191759519, "map@50": 0.01480552152725842, "precision": 0.047619047619047616, "recall": 0.06666666666666667, "f1_score": 0.05555555555555555}, {"class": "Haul Truck", "map@50:95": 0.5163962780515491, "map@50": 0.9306196364918045, "precision": 0.8809523809523809, "recall": 0.8809523809523809, "f1_score": 0.8809523809523809}, {"class": "Helicopter", "map@50:95": 0.005302628477133428, "map@50": 0.02601956624233852, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Helipad", "map@50:95": 0.006886917139989862, "map@50": 0.01306682392377169, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Hut-Tent", "map@50:95": 3.1835981025755303e-05, "map@50": 7.640635446181274e-05, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Locomotive", "map@50:95": 0.0, "map@50": 0.0, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Maritime Vessel", "map@50:95": 0.07448085016229791, "map@50": 0.13869505084699524, "precision": 0.30434782608695654, "recall": 0.175, "f1_score": 0.2222222222222222}, {"class": "Mobile Crane", "map@50:95": 0.008832174444126498, "map@50": 0.017502004633848253, "precision": 0.06666666666666667, "recall": 0.0196078431372549, "f1_score": 0.0303030303030303}, {"class": "Motorboat", "map@50:95": 0.09639133777947154, "map@50": 0.28820619102988965, "precision": 0.23130841121495327, "recall": 0.5963855421686747, "f1_score": 0.33333333333333337}, {"class": "Oil Tanker", "map@50:95": 0.12842462301423904, "map@50": 0.2350286714563203, "precision": 0.3333333333333333, "recall": 0.16666666666666666, "f1_score": 0.2222222222222222}, {"class": "Passenger Car", "map@50:95": 0.09503784273507607, "map@50": 0.2100902729853239, "precision": 0.8, "recall": 0.11149825783972125, "f1_score": 0.19571865443425077}, {"class": "Passenger Vehicle", "map@50:95": 0.0035259441934329034, "map@50": 0.012619653407360671, "precision": 0.01894150417827298, "recall": 0.06390977443609022, "f1_score": 0.029222174473571123}, {"class": "Pickup Truck", "map@50:95": 0.0004398640433710404, "map@50": 0.0014540498773397011, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Pylon", "map@50:95": 0.22472526269755266, "map@50": 0.4490759387270336, "precision": 0.41333333333333333, "recall": 0.45588235294117646, "f1_score": 0.43356643356643354}, {"class": "Reach Stacker", "map@50:95": 3.2705973300032705e-05, "map@50": 0.00032705973300032703, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Sailboat", "map@50:95": 0.18248338420428073, "map@50": 0.41474227461749424, "precision": 0.2774869109947644, "recall": 0.726027397260274, "f1_score": 0.40151515151515155}, {"class": "Scraper-Tractor", "map@50:95": 0.00021932573004135861, "map@50": 0.0003133224714876551, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Shed", "map@50:95": 0.021252346559502748, "map@50": 0.04608291999964602, "precision": 0.11078717201166181, "recall": 0.12794612794612795, "f1_score": 0.11875000000000001}, {"class": "Shipping Container", "map@50:95": 0.03667908763148507, "map@50": 0.07442284603692942, "precision": 0.17010309278350516, "recall": 0.14410480349344978, "f1_score": 0.15602836879432624}, {"class": "Shipping container lot", "map@50:95": 0.09281775834476118, "map@50": 0.19995557899737745, "precision": 0.15422885572139303, "recall": 0.5375722543352601, "f1_score": 0.23969072164948454}, {"class": "Small Aircraft", "map@50:95": 0.3364730683803633, "map@50": 0.6399959993734593, "precision": 0.30434782608695654, "recall": 0.7777777777777778, "f1_score": 0.43750000000000006}, {"class": "Small Car", "map@50:95": 0.06647212340434179, "map@50": 0.2494447400504334, "precision": 0.27931270487374377, "recall": 0.39419435194834906, "f1_score": 0.3269557150838578}, {"class": "Storage Tank", "map@50:95": 0.29185975675745157, "map@50": 0.5234221637202883, "precision": 0.46174142480211083, "recall": 0.5663430420711975, "f1_score": 0.5087209302325583}, {"class": "Straddle Carrier", "map@50:95": 0.00018111567254286407, "map@50": 0.0006037189084762136, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tank car", "map@50:95": 0.0, "map@50": 0.0, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tower", "map@50:95": 0.0, "map@50": 0.0, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tower crane", "map@50:95": 0.004293009032393638, "map@50": 0.00859510411472802, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Trailer", "map@50:95": 0.0197718469392616, "map@50": 0.05170822103312767, "precision": 0.16552667578659372, "recall": 0.1662087912087912, "f1_score": 0.1658670322138451}, {"class": "Truck", "map@50:95": 0.02843881366567936, "map@50": 0.0720059440374855, "precision": 0.09276574803149606, "recall": 0.3165407220822838, "f1_score": 0.1434823977164605}, {"class": "Truck Tractor", "map@50:95": 0.0007162555785911271, "map@50": 0.0014370321572274643, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Truck w-Box", "map@50:95": 0.058296666326410784, "map@50": 0.14736265647769678, "precision": 0.26863572433192684, "recall": 0.2496732026143791, "f1_score": 0.2588075880758807}, {"class": "Truck w-Flatbed", "map@50:95": 0.007886342271246292, "map@50": 0.020914181932651967, "precision": 0.03511852502194908, "recall": 0.272108843537415, "f1_score": 0.062208398133748066}, {"class": "Truck w-Liquid", "map@50:95": 0.0, "map@50": 0.0, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tugboat", "map@50:95": 0.05403520528841395, "map@50": 0.11284311309872196, "precision": 0.06735751295336788, "recall": 0.40625, "f1_score": 0.11555555555555555}, {"class": "Utility Truck", "map@50:95": 0.0025805972531610993, "map@50": 0.008249394932244785, "precision": 0.09615384615384616, "recall": 0.01935483870967742, "f1_score": 0.0322234156820623}, {"class": "Vehicle Lot", "map@50:95": 0.006628153266717578, "map@50": 0.024170207377052267, "precision": 0.0899581589958159, "recall": 0.07037643207855974, "f1_score": 0.07897153351698807}, {"class": "Yacht", "map@50:95": 0.23584914708878507, "map@50": 0.49655454385963366, "precision": 0.5652173913043478, "recall": 0.5098039215686274, "f1_score": 0.5360824742268041}, {"class": "all", "map@50:95": 0.0757745315258965, "map@50": 0.15195648033589385, "precision": 0.17435226252670177, "recall": 0.2191285895303374, "f1_score": 0.16264379972958676}], "map": 0.15195648033589385, "precision": 0.17435226252670177, "recall": 0.2191285895303374, "f1_score": 0.16264379972958676}, "test_coco_eval_bbox": [0.0757745315258965, 0.15195648033589385, 0.06701869929551499, 0.047116486764487986, 0.09448557038338687, 0.1400829641575825, 0.06278241030514746, 0.15973483760669976, 0.19558511310010426, 0.12231404842224113, 0.24495376465507418, 0.33507023424136717], "epoch": 0, "n_parameters": 126141694, "best_res": 0.0757745315258965, "best_ep": 0, "now_time": "2026-02-01 06:08:24.027894", "train_epoch_time": "1:10:14", "epoch_time": "1:16:51"}
{"train_lr": 9.999999999999097e-05, "train_class_error": 23.01311798132587, "train_loss": 11.390376459346871, "train_loss_ce": 0.7919005264882604, "train_loss_bbox": 0.18671850917749655, "train_loss_giou": 0.901699005416555, "train_loss_ce_0": 0.7952463098177582, "train_loss_bbox_0": 0.1940323542769758, "train_loss_giou_0": 0.9233421573011533, "train_loss_ce_1": 0.7896737719222238, "train_loss_bbox_1": 0.19107506232296093, "train_loss_giou_1": 0.9152755372879987, "train_loss_ce_2": 0.7910108121355626, "train_loss_bbox_2": 0.18840663937605506, "train_loss_giou_2": 0.908561276991874, "train_loss_ce_3": 0.7916548083982515, "train_loss_bbox_3": 0.18709971775486076, "train_loss_giou_3": 0.9039832155522097, "train_loss_ce_enc": 0.7774125413592308, "train_loss_bbox_enc": 0.20314257991911494, "train_loss_giou_enc": 0.9501416302230484, "train_loss_ce_unscaled": 0.7919005264882604, "train_class_error_unscaled": 23.01311798132587, "train_loss_bbox_unscaled": 0.037343701825413925, "train_loss_giou_unscaled": 0.4508495027082775, "train_cardinality_error_unscaled": 3834.779308677786, "train_loss_ce_0_unscaled": 0.7952463098177582, "train_loss_bbox_0_unscaled": 0.03880647086304081, "train_loss_giou_0_unscaled": 0.46167107865057666, "train_cardinality_error_0_unscaled": 3834.726250906454, "train_loss_ce_1_unscaled": 0.7896737719222238, "train_loss_bbox_1_unscaled": 0.0382150124842283, "train_loss_giou_1_unscaled": 0.45763776864399935, "train_cardinality_error_1_unscaled": 3835.126057529611, "train_loss_ce_2_unscaled": 0.7910108121355626, "train_loss_bbox_2_unscaled": 0.037681327860915874, "train_loss_giou_2_unscaled": 0.454280638495937, "train_cardinality_error_2_unscaled": 3834.894730481025, "train_loss_ce_3_unscaled": 0.7916548083982515, "train_loss_bbox_3_unscaled": 0.03741994356783929, "train_loss_giou_3_unscaled": 0.45199160777610486, "train_cardinality_error_3_unscaled": 3834.851099830795, "train_loss_ce_enc_unscaled": 0.7774125413592308, "train_loss_bbox_enc_unscaled": 0.04062851599969115, "train_loss_giou_enc_unscaled": 0.4750708151115242, "train_cardinality_error_enc_unscaled": 3834.5383127870437, "test_class_error": 20.329072827535963, "test_loss": 10.830671586687602, "test_loss_ce": 0.8176224578373016, "test_loss_bbox": 0.15063377148220464, "test_loss_giou": 0.8189313829985876, "test_loss_ce_0": 0.8184802827380953, "test_loss_bbox_0": 0.15706083557553707, "test_loss_giou_0": 0.8387830022687004, "test_loss_ce_1": 0.813421378968254, "test_loss_bbox_1": 0.15511305176076434, "test_loss_giou_1": 0.8327834580436585, "test_loss_ce_2": 0.8149844990079365, "test_loss_bbox_2": 0.15319037914276123, "test_loss_giou_2": 0.8266943950123257, "test_loss_ce_3": 0.815750248015873, "test_loss_bbox_3": 0.15218768538463684, "test_loss_giou_3": 0.8205757940667017, "test_loss_ce_enc": 0.8037109375, "test_loss_bbox_enc": 0.1675135745746749, "test_loss_giou_enc": 0.8732344519335126, "test_loss_ce_unscaled": 0.8176224578373016, "test_class_error_unscaled": 20.329072827535963, "test_loss_bbox_unscaled": 0.030126754315569996, "test_loss_giou_unscaled": 0.4094656914992938, "test_cardinality_error_unscaled": 236.68, "test_loss_ce_0_unscaled": 0.8184802827380953, "test_loss_bbox_0_unscaled": 0.0314121671099334, "test_loss_giou_0_unscaled": 0.4193915011343502, "test_cardinality_error_0_unscaled": 236.68698412698413, "test_loss_ce_1_unscaled": 0.813421378968254, "test_loss_bbox_1_unscaled": 0.031022610363092215, "test_loss_giou_1_unscaled": 0.41639172902182925, "test_cardinality_error_1_unscaled": 236.70984126984126, "test_loss_ce_2_unscaled": 0.8149844990079365, "test_loss_bbox_2_unscaled": 0.03063807584448821, "test_loss_giou_2_unscaled": 0.41334719750616283, "test_cardinality_error_2_unscaled": 236.71174603174603, "test_loss_ce_3_unscaled": 0.815750248015873, "test_loss_bbox_3_unscaled": 0.030437537055492164, "test_loss_giou_3_unscaled": 0.41028789703335083, "test_cardinality_error_3_unscaled": 236.6431746031746, "test_loss_ce_enc_unscaled": 0.8037109375, "test_loss_bbox_enc_unscaled": 0.0335027149356606, "test_loss_giou_enc_unscaled": 0.4366172259667563, "test_cardinality_error_enc_unscaled": 236.62349206349205, "test_results_json": {"class_map": [{"class": "Aircraft Hangar", "map@50:95": 0.01348769607499672, "map@50": 0.020650508164588913, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Barge", "map@50:95": 0.06028993856335859, "map@50": 0.12327004913882665, "precision": 0.3333333333333333, "recall": 0.06818181818181818, "f1_score": 0.11320754716981131}, {"class": "Building", "map@50:95": 0.23963928300680007, "map@50": 0.5624700779001185, "precision": 0.4703203868644915, "recall": 0.6763048811437272, "f1_score": 0.5548106817677104}, {"class": "Bus", "map@50:95": 0.07069755624790727, "map@50": 0.1849996069336509, "precision": 0.2880466472303207, "recall": 0.3491166077738516, "f1_score": 0.3156549520766773}, {"class": "Cargo Car", "map@50:95": 0.13768198680319105, "map@50": 0.2871250915991303, "precision": 0.23950617283950618, "recall": 0.7348484848484849, "f1_score": 0.3612662942271881}, {"class": "Cargo Plane", "map@50:95": 0.4817220036506115, "map@50": 0.8173608087183137, "precision": 0.7058823529411765, "recall": 0.8089887640449438, "f1_score": 0.7539267015706808}, {"class": "Cargo Truck", "map@50:95": 0.013005504818601465, "map@50": 0.042269698355467074, "precision": 0.09009254218835057, "recall": 0.28559102674719583, "f1_score": 0.13697496379060625}, {"class": "Cement Mixer", "map@50:95": 0.050410703016711876, "map@50": 0.15750157938648604, "precision": 0.2857142857142857, "recall": 0.18181818181818182, "f1_score": 0.2222222222222222}, {"class": "Construction Site", "map@50:95": 0.0009631385239729711, "map@50": 0.0037598545827607576, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Container Crane", "map@50:95": 0.0722757618416578, "map@50": 0.10855645423697298, "precision": 0.11904761904761904, "recall": 0.2631578947368421, "f1_score": 0.1639344262295082}, {"class": "Container Ship", "map@50:95": 0.38485099953880236, "map@50": 0.5351978875305979, "precision": 0.4507042253521127, "recall": 0.6153846153846154, "f1_score": 0.5203252032520326}, {"class": "Crane Truck", "map@50:95": 0.0013437346532345139, "map@50": 0.0035119266429178325, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Damaged Building", "map@50:95": 0.05237140583672058, "map@50": 0.11290437803517629, "precision": 0.21755725190839695, "recall": 0.20430107526881722, "f1_score": 0.21072088724584104}, {"class": "Dump Truck", "map@50:95": 0.05434791458569964, "map@50": 0.12250133799812978, "precision": 0.20952380952380953, "recall": 0.2214765100671141, "f1_score": 0.21533442088091354}, {"class": "Engineering Vehicle", "map@50:95": 0.0428880088533029, "map@50": 0.09524074656404567, "precision": 0.16129032258064516, "recall": 0.1111111111111111, "f1_score": 0.13157894736842105}, {"class": "Excavator", "map@50:95": 0.12880138263402852, "map@50": 0.2989288108445226, "precision": 0.29411764705882354, "recall": 0.38461538461538464, "f1_score": 0.33333333333333337}, {"class": "Facility", "map@50:95": 0.07781518657144512, "map@50": 0.13524058877310835, "precision": 0.13358778625954199, "recall": 0.3211009174311927, "f1_score": 0.18867924528301885}, {"class": "Ferry", "map@50:95": 0.08512846281431066, "map@50": 0.14503786496756155, "precision": 0.16666666666666666, "recall": 0.20833333333333334, "f1_score": 0.1851851851851852}, {"class": "Fishing Vessel", "map@50:95": 0.044994415395514895, "map@50": 0.09458906138201183, "precision": 0.2459016393442623, "recall": 0.19480519480519481, "f1_score": 0.21739130434782608}, {"class": "Fixed-wing Aircraft", "map@50:95": 0.06501799737482633, "map@50": 0.17169522638051327, "precision": 0.16666666666666666, "recall": 0.13636363636363635, "f1_score": 0.15}, {"class": "Flat Car", "map@50:95": 0.0, "map@50": 0.0, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Front loader-Bulldozer", "map@50:95": 0.04547615551596286, "map@50": 0.11293299394792389, "precision": 0.10056925996204934, "recall": 0.37857142857142856, "f1_score": 0.15892053973013495}, {"class": "Ground Grader", "map@50:95": 0.06015761176580072, "map@50": 0.0964569179136714, "precision": 0.045454545454545456, "recall": 0.13333333333333333, "f1_score": 0.06779661016949153}, {"class": "Haul Truck", "map@50:95": 0.5139687992771169, "map@50": 0.9295467379592974, "precision": 0.8142857142857143, "recall": 0.9047619047619048, "f1_score": 0.857142857142857}, {"class": "Helicopter", "map@50:95": 0.013012729844413012, "map@50": 0.05198019801980199, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Helipad", "map@50:95": 0.04736210677923085, "map@50": 0.07755080490326162, "precision": 0.13636363636363635, "recall": 0.125, "f1_score": 0.13043478260869565}, {"class": "Hut-Tent", "map@50:95": 0.00025995797306689497, "map@50": 0.000648821327148351, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Locomotive", "map@50:95": 0.028399210215746484, "map@50": 0.05534094861126174, "precision": 0.06329113924050633, "recall": 0.2631578947368421, "f1_score": 0.10204081632653061}, {"class": "Maritime Vessel", "map@50:95": 0.08864155258645867, "map@50": 0.1896040569057278, "precision": 0.12034383954154727, "recall": 0.525, "f1_score": 0.19580419580419584}, {"class": "Mobile Crane", "map@50:95": 0.01606574388737992, "map@50": 0.03556047771040302, "precision": 0.1111111111111111, "recall": 0.1568627450980392, "f1_score": 0.13008130081300812}, {"class": "Motorboat", "map@50:95": 0.10136928054536151, "map@50": 0.3008510929850126, "precision": 0.4827586206896552, "recall": 0.25301204819277107, "f1_score": 0.3320158102766798}, {"class": "Oil Tanker", "map@50:95": 0.19796838099492042, "map@50": 0.3795060885398885, "precision": 0.125, "recall": 0.3333333333333333, "f1_score": 0.18181818181818182}, {"class": "Passenger Car", "map@50:95": 0.27958999161476283, "map@50": 0.6908272023201617, "precision": 0.6371951219512195, "recall": 0.7282229965156795, "f1_score": 0.6796747967479675}, {"class": "Passenger Vehicle", "map@50:95": 0.0017160206752368668, "map@50": 0.0071925611543788865, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Pickup Truck", "map@50:95": 0.0008795017808716171, "map@50": 0.002188924888625005, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Pylon", "map@50:95": 0.25539301197228703, "map@50": 0.4676952139626492, "precision": 0.5303030303030303, "recall": 0.5147058823529411, "f1_score": 0.5223880597014925}, {"class": "Reach Stacker", "map@50:95": 0.0, "map@50": 0.0, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Sailboat", "map@50:95": 0.18763097318718497, "map@50": 0.41602494543938945, "precision": 0.5555555555555556, "recall": 0.3424657534246575, "f1_score": 0.42372881355932196}, {"class": "Scraper-Tractor", "map@50:95": 0.008696675498261999, "map@50": 0.011991974278409954, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Shed", "map@50:95": 0.0626884608901683, "map@50": 0.1290720333684921, "precision": 0.6470588235294118, "recall": 0.07407407407407407, "f1_score": 0.13293051359516617}, {"class": "Shipping Container", "map@50:95": 0.057408274590236286, "map@50": 0.1132275092365565, "precision": 0.21428571428571427, "recall": 0.1572052401746725, "f1_score": 0.18136020151133503}, {"class": "Shipping container lot", "map@50:95": 0.12910904959647676, "map@50": 0.2725193173736317, "precision": 0.25483870967741934, "recall": 0.45664739884393063, "f1_score": 0.3271221532091097}, {"class": "Small Aircraft", "map@50:95": 0.32289406602359433, "map@50": 0.6204080226537554, "precision": 0.3401360544217687, "recall": 0.7936507936507936, "f1_score": 0.4761904761904761}, {"class": "Small Car", "map@50:95": 0.07616380822847522, "map@50": 0.29001427869821234, "precision": 0.3468404647208841, "recall": 0.43557611651966144, "f1_score": 0.3861764341329848}, {"class": "Storage Tank", "map@50:95": 0.24947633291163362, "map@50": 0.4852113504218713, "precision": 0.7225130890052356, "recall": 0.44660194174757284, "f1_score": 0.552}, {"class": "Straddle Carrier", "map@50:95": 0.00225198990487284, "map@50": 0.005629974762182101, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tank car", "map@50:95": 0.0, "map@50": 0.0, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tower", "map@50:95": 0.0, "map@50": 0.0, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tower crane", "map@50:95": 0.0030733055606445596, "map@50": 0.008712871287128712, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Trailer", "map@50:95": 0.03757408395094779, "map@50": 0.09963444852960945, "precision": 0.1165230423379543, "recall": 0.4271978021978022, "f1_score": 0.18310273770974392}, {"class": "Truck", "map@50:95": 0.03085934701220493, "map@50": 0.07982241420316638, "precision": 0.15445544554455445, "recall": 0.22921914357682618, "f1_score": 0.18455298293053912}, {"class": "Truck Tractor", "map@50:95": 0.0003839469397014641, "map@50": 0.001959351720170499, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Truck w-Box", "map@50:95": 0.07634551893045309, "map@50": 0.18500391330253713, "precision": 0.21216509775524983, "recall": 0.38300653594771245, "f1_score": 0.27306616961789376}, {"class": "Truck w-Flatbed", "map@50:95": 0.005731810924578206, "map@50": 0.014654472841615019, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Truck w-Liquid", "map@50:95": 0.034739029831461, "map@50": 0.07454913951579074, "precision": 0.42857142857142855, "recall": 0.07894736842105263, "f1_score": 0.13333333333333333}, {"class": "Tugboat", "map@50:95": 0.08404240012944833, "map@50": 0.20345462268991635, "precision": 0.1794871794871795, "recall": 0.4375, "f1_score": 0.2545454545454545}, {"class": "Utility Truck", "map@50:95": 0.005689542513352797, "map@50": 0.015516065099389333, "precision": 0.10309278350515463, "recall": 0.012903225806451613, "f1_score": 0.02293577981651376}, {"class": "Vehicle Lot", "map@50:95": 0.007713363617723205, "map@50": 0.026661434084099742, "precision": 0.07342657342657342, "recall": 0.10310965630114566, "f1_score": 0.08577263444520081}, {"class": "Yacht", "map@50:95": 0.31815051405888356, "map@50": 0.6338670200287454, "precision": 0.29931972789115646, "recall": 0.8627450980392157, "f1_score": 0.44444444444444453}, {"class": "all", "map@50:95": 0.09201043441584092, "map@50": 0.18828186031896244, "precision": 0.2100492383752248, "recall": 0.2597001890389371, "f1_score": 0.2066767185785039}], "map": 0.18828186031896244, "precision": 0.2100492383752248, "recall": 0.2597001890389371, "f1_score": 0.2066767185785039}, "test_coco_eval_bbox": [0.09201043441584092, 0.18828186031896244, 0.08024957500227081, 0.060934249014587674, 0.12906677756549875, 0.16586563760384163, 0.07267616166406757, 0.1906823559286815, 0.2400612682247823, 0.16899901278902849, 0.3070182919463558, 0.35991174271213827], "epoch": 1, "n_parameters": 126141694, "best_res": 0.09201043441584092, "best_ep": 1, "now_time": "2026-02-01 07:25:30.293210", "train_epoch_time": "1:10:19", "epoch_time": "1:17:06"}
{"train_lr": 9.999999999999097e-05, "train_class_error": 20.09012043081334, "train_loss": 11.017061353984618, "train_loss_ce": 0.7758392888861503, "train_loss_bbox": 0.17467870035406644, "train_loss_giou": 0.8694388295078309, "train_loss_ce_0": 0.7762013976694622, "train_loss_bbox_0": 0.18155689235458794, "train_loss_giou_0": 0.8900797353822856, "train_loss_ce_1": 0.7717601361927406, "train_loss_bbox_1": 0.1786591938764563, "train_loss_giou_1": 0.8831326761988733, "train_loss_ce_2": 0.7737311080945674, "train_loss_bbox_2": 0.17641448870322407, "train_loss_giou_2": 0.8756626785793813, "train_loss_ce_3": 0.7747915545082533, "train_loss_bbox_3": 0.1751559834389508, "train_loss_giou_3": 0.8715489580272793, "train_loss_ce_enc": 0.7621252275474046, "train_loss_bbox_enc": 0.1901773455403502, "train_loss_giou_enc": 0.9161071570089021, "train_loss_ce_unscaled": 0.7758392888861503, "train_class_error_unscaled": 20.09012043081334, "train_loss_bbox_unscaled": 0.03493574008320616, "train_loss_giou_unscaled": 0.43471941475391546, "train_cardinality_error_unscaled": 3833.377810007252, "train_loss_ce_0_unscaled": 0.7762013976694622, "train_loss_bbox_0_unscaled": 0.03631137848383105, "train_loss_giou_0_unscaled": 0.4450398676911428, "train_cardinality_error_0_unscaled": 3833.602852308436, "train_loss_ce_1_unscaled": 0.7717601361927406, "train_loss_bbox_1_unscaled": 0.035731838773470595, "train_loss_giou_1_unscaled": 0.44156633809943663, "train_cardinality_error_1_unscaled": 3833.78233019096, "train_loss_ce_2_unscaled": 0.7737311080945674, "train_loss_bbox_2_unscaled": 0.03528289773634783, "train_loss_giou_2_unscaled": 0.43783133928969065, "train_cardinality_error_2_unscaled": 3833.676335508823, "train_loss_ce_3_unscaled": 0.7747915545082533, "train_loss_bbox_3_unscaled": 0.035031196695117825, "train_loss_giou_3_unscaled": 0.43577447901363964, "train_cardinality_error_3_unscaled": 3833.2368866328256, "train_loss_ce_enc_unscaled": 0.7621252275474046, "train_loss_bbox_enc_unscaled": 0.038035469133438284, "train_loss_giou_enc_unscaled": 0.45805357850445105, "train_cardinality_error_enc_unscaled": 3833.375151075659, "test_class_error": 19.71938035026429, "test_loss": 10.765020831880115, "test_loss_ce": 0.8106398809523809, "test_loss_bbox": 0.1472100209432935, "test_loss_giou": 0.813969823557233, "test_loss_ce_0": 0.8123573908730158, "test_loss_bbox_0": 0.15463876680249258, "test_loss_giou_0": 0.8370822042699844, "test_loss_ce_1": 0.8100384424603174, "test_loss_bbox_1": 0.15161243139278321, "test_loss_giou_1": 0.8297021639820129, "test_loss_ce_2": 0.8075892857142857, "test_loss_bbox_2": 0.15008402573210852, "test_loss_giou_2": 0.8244724951471601, "test_loss_ce_3": 0.808992435515873, "test_loss_bbox_3": 0.1487873322895122, "test_loss_giou_3": 0.8167542054255803, "test_loss_ce_enc": 0.7993551587301587, "test_loss_bbox_enc": 0.16887977051947797, "test_loss_giou_enc": 0.8728549824631403, "test_loss_ce_unscaled": 0.8106398809523809, "test_class_error_unscaled": 19.71938035026429, "test_loss_bbox_unscaled": 0.029442004233096092, "test_loss_giou_unscaled": 0.4069849117786165, "test_cardinality_error_unscaled": 236.71809523809523, "test_loss_ce_0_unscaled": 0.8123573908730158, "test_loss_bbox_0_unscaled": 0.030927753373093548, "test_loss_giou_0_unscaled": 0.4185411021349922, "test_cardinality_error_0_unscaled": 236.75746031746033, "test_loss_ce_1_unscaled": 0.8100384424603174, "test_loss_bbox_1_unscaled": 0.030322486269509508, "test_loss_giou_1_unscaled": 0.41485108199100645, "test_cardinality_error_1_unscaled": 236.77396825396826, "test_loss_ce_2_unscaled": 0.8075892857142857, "test_loss_bbox_2_unscaled": 0.030016805168300395, "test_loss_giou_2_unscaled": 0.4122362475735801, "test_cardinality_error_2_unscaled": 236.7631746031746, "test_loss_ce_3_unscaled": 0.808992435515873, "test_loss_bbox_3_unscaled": 0.0297574664139381, "test_loss_giou_3_unscaled": 0.40837710271279015, "test_cardinality_error_3_unscaled": 236.73650793650793, "test_loss_ce_enc_unscaled": 0.7993551587301587, "test_loss_bbox_enc_unscaled": 0.03377595413875367, "test_loss_giou_enc_unscaled": 0.43642749123157015, "test_cardinality_error_enc_unscaled": 236.7295238095238, "test_results_json": {"class_map": [{"class": "Aircraft Hangar", "map@50:95": 0.035184345740179805, "map@50": 0.05224035061734021, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Barge", "map@50:95": 0.23438827744729487, "map@50": 0.37413298310964305, "precision": 0.2696629213483146, "recall": 0.5454545454545454, "f1_score": 0.3609022556390977}, {"class": "Building", "map@50:95": 0.2434729848795107, "map@50": 0.5618546956727483, "precision": 0.5231875117345707, "recall": 0.660045681414432, "f1_score": 0.5837017752541689}, {"class": "Bus", "map@50:95": 0.09437409143237402, "map@50": 0.23169979974087024, "precision": 0.23398643556895252, "recall": 0.43886925795053006, "f1_score": 0.3052347014008356}, {"class": "Cargo Car", "map@50:95": 0.23867078864402597, "map@50": 0.520268793689151, "precision": 0.09636517328825021, "recall": 0.8636363636363636, "f1_score": 0.17338403041825096}, {"class": "Cargo Plane", "map@50:95": 0.5038199423760784, "map@50": 0.8375996395062152, "precision": 0.5306122448979592, "recall": 0.8764044943820225, "f1_score": 0.6610169491525424}, {"class": "Cargo Truck", "map@50:95": 0.02207551478330813, "map@50": 0.06425713176249327, "precision": 0.10558605101711334, "recall": 0.28213977566867987, "f1_score": 0.1536654135338346}, {"class": "Cement Mixer", "map@50:95": 0.07533978116649652, "map@50": 0.22311642604005733, "precision": 0.6111111111111112, "recall": 0.2, "f1_score": 0.30136986301369867}, {"class": "Construction Site", "map@50:95": 0.001399143789614371, "map@50": 0.00495448194609598, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Container Crane", "map@50:95": 0.09126804411825376, "map@50": 0.17365941139568503, "precision": 0.21428571428571427, "recall": 0.15789473684210525, "f1_score": 0.18181818181818182}, {"class": "Container Ship", "map@50:95": 0.3128930882165983, "map@50": 0.47491812578329395, "precision": 0.2518518518518518, "recall": 0.6538461538461539, "f1_score": 0.3636363636363636}, {"class": "Crane Truck", "map@50:95": 0.005700230277790607, "map@50": 0.012104493516866769, "precision": 0.07407407407407407, "recall": 0.08695652173913043, "f1_score": 0.07999999999999999}, {"class": "Damaged Building", "map@50:95": 0.04596384794992101, "map@50": 0.10879707506663476, "precision": 0.3246753246753247, "recall": 0.17921146953405018, "f1_score": 0.23094688221709006}, {"class": "Dump Truck", "map@50:95": 0.05291445016987413, "map@50": 0.10228939768412368, "precision": 0.28421052631578947, "recall": 0.09060402684563758, "f1_score": 0.13740458015267176}, {"class": "Engineering Vehicle", "map@50:95": 0.04640556195133996, "map@50": 0.08531854860909713, "precision": 0.20689655172413793, "recall": 0.13333333333333333, "f1_score": 0.16216216216216217}, {"class": "Excavator", "map@50:95": 0.13351391375775973, "map@50": 0.3261844636663985, "precision": 0.23809523809523808, "recall": 0.43956043956043955, "f1_score": 0.3088803088803088}, {"class": "Facility", "map@50:95": 0.10153958131994834, "map@50": 0.19609941308276238, "precision": 0.3333333333333333, "recall": 0.23853211009174313, "f1_score": 0.27807486631016043}, {"class": "Ferry", "map@50:95": 0.08024424623524491, "map@50": 0.14983716514801512, "precision": 0.22641509433962265, "recall": 0.5, "f1_score": 0.31168831168831174}, {"class": "Fishing Vessel", "map@50:95": 0.05815400011227412, "map@50": 0.13366156441276783, "precision": 0.192, "recall": 0.3116883116883117, "f1_score": 0.2376237623762376}, {"class": "Fixed-wing Aircraft", "map@50:95": 0.0726090740942226, "map@50": 0.21800311899321798, "precision": 0.5, "recall": 0.09090909090909091, "f1_score": 0.15384615384615385}, {"class": "Flat Car", "map@50:95": 0.0024827122568199197, "map@50": 0.006494659069748512, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Front loader-Bulldozer", "map@50:95": 0.041336177097989826, "map@50": 0.12431188819764455, "precision": 0.14285714285714285, "recall": 0.29285714285714287, "f1_score": 0.19203747072599534}, {"class": "Ground Grader", "map@50:95": 0.0177845028745274, "map@50": 0.029948150631509487, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Haul Truck", "map@50:95": 0.5403918921355991, "map@50": 0.933711175988175, "precision": 0.7300613496932515, "recall": 0.9444444444444444, "f1_score": 0.8235294117647058}, {"class": "Helicopter", "map@50:95": 0.012541254125412543, "map@50": 0.08283828382838285, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Helipad", "map@50:95": 0.04780382597530269, "map@50": 0.10069718218022408, "precision": 0.5, "recall": 0.041666666666666664, "f1_score": 0.07692307692307693}, {"class": "Hut-Tent", "map@50:95": 0.00048542785582153335, "map@50": 0.0012887262033079636, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Locomotive", "map@50:95": 0.04684512931087434, "map@50": 0.08978591753314574, "precision": 0.07291666666666667, "recall": 0.3684210526315789, "f1_score": 0.12173913043478261}, {"class": "Maritime Vessel", "map@50:95": 0.10677035140848443, "map@50": 0.18951280867463546, "precision": 0.13486842105263158, "recall": 0.5125, "f1_score": 0.21354166666666663}, {"class": "Mobile Crane", "map@50:95": 0.031990161623562954, "map@50": 0.05993720681895968, "precision": 0.15217391304347827, "recall": 0.13725490196078433, "f1_score": 0.1443298969072165}, {"class": "Motorboat", "map@50:95": 0.11249845646179606, "map@50": 0.3335788903114254, "precision": 0.5647058823529412, "recall": 0.2891566265060241, "f1_score": 0.3824701195219123}, {"class": "Oil Tanker", "map@50:95": 0.159578791319048, "map@50": 0.35344678944683183, "precision": 0.10526315789473684, "recall": 0.3333333333333333, "f1_score": 0.16}, {"class": "Passenger Car", "map@50:95": 0.3509458091384391, "map@50": 0.737088496265682, "precision": 0.5216450216450217, "recall": 0.8397212543554007, "f1_score": 0.643524699599466}, {"class": "Passenger Vehicle", "map@50:95": 0.0008035926917626536, "map@50": 0.002852858475780967, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Pickup Truck", "map@50:95": 0.0010818133610562765, "map@50": 0.0034099326993434717, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Pylon", "map@50:95": 0.282676570896211, "map@50": 0.506717888261867, "precision": 0.6071428571428571, "recall": 0.5, "f1_score": 0.5483870967741935}, {"class": "Reach Stacker", "map@50:95": 0.0, "map@50": 0.0, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Sailboat", "map@50:95": 0.14633434524059757, "map@50": 0.32238383470929804, "precision": 0.26436781609195403, "recall": 0.6301369863013698, "f1_score": 0.3724696356275304}, {"class": "Scraper-Tractor", "map@50:95": 0.0014916785796226684, "map@50": 0.0027720419100733607, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Shed", "map@50:95": 0.06751852524243954, "map@50": 0.13319136657096947, "precision": 0.47692307692307695, "recall": 0.10437710437710437, "f1_score": 0.1712707182320442}, {"class": "Shipping Container", "map@50:95": 0.05919202418657545, "map@50": 0.11317247843830759, "precision": 0.15548780487804878, "recall": 0.22270742358078602, "f1_score": 0.18312387791741472}, {"class": "Shipping container lot", "map@50:95": 0.09420089609762453, "map@50": 0.21243484711958988, "precision": 0.14592592592592593, "recall": 0.569364161849711, "f1_score": 0.23231132075471697}, {"class": "Small Aircraft", "map@50:95": 0.2781508052803681, "map@50": 0.5766509056489169, "precision": 0.5060240963855421, "recall": 0.6666666666666666, "f1_score": 0.5753424657534246}, {"class": "Small Car", "map@50:95": 0.0790032956579729, "map@50": 0.297963143810816, "precision": 0.3850603393521459, "recall": 0.43148369385628227, "f1_score": 0.40695235241234645}, {"class": "Storage Tank", "map@50:95": 0.27375852107339965, "map@50": 0.5221274348114479, "precision": 0.5522875816993464, "recall": 0.5469255663430421, "f1_score": 0.5495934959349594}, {"class": "Straddle Carrier", "map@50:95": 0.0, "map@50": 0.0, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tank car", "map@50:95": 0.0, "map@50": 0.0, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tower", "map@50:95": 0.014823196605374822, "map@50": 0.031117397454031116, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tower crane", "map@50:95": 0.0014736357356665898, "map@50": 0.0036840893391664746, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Trailer", "map@50:95": 0.030184039891741517, "map@50": 0.07937223240383297, "precision": 0.32666666666666666, "recall": 0.0673076923076923, "f1_score": 0.11161731207289294}, {"class": "Truck", "map@50:95": 0.03148157270555157, "map@50": 0.07679515227396497, "precision": 0.17418546365914786, "recall": 0.17506297229219145, "f1_score": 0.17462311557788945}, {"class": "Truck Tractor", "map@50:95": 0.000476672554811703, "map@50": 0.001188118811881188, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Truck w-Box", "map@50:95": 0.11922251667301878, "map@50": 0.3078918506363923, "precision": 0.25015873015873014, "recall": 0.515032679738562, "f1_score": 0.3367521367521367}, {"class": "Truck w-Flatbed", "map@50:95": 0.014881322474180367, "map@50": 0.03628000630779994, "precision": 0.13592233009708737, "recall": 0.09523809523809523, "f1_score": 0.11199999999999999}, {"class": "Truck w-Liquid", "map@50:95": 0.00014704440741103813, "map@50": 0.00029408881482207626, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tugboat", "map@50:95": 0.0706058468405433, "map@50": 0.16092732715557576, "precision": 0.19480519480519481, "recall": 0.46875, "f1_score": 0.27522935779816515}, {"class": "Utility Truck", "map@50:95": 0.005509954026936933, "map@50": 0.019037334874290456, "precision": 0.13131313131313133, "recall": 0.016774193548387096, "f1_score": 0.029748283752860406}, {"class": "Vehicle Lot", "map@50:95": 0.006806745880133936, "map@50": 0.023787967889277027, "precision": 0.12085308056872038, "recall": 0.08346972176759411, "f1_score": 0.09874152952565343}, {"class": "Yacht", "map@50:95": 0.311807026357217, "map@50": 0.624312008825483, "precision": 0.26900584795321636, "recall": 0.9019607843137255, "f1_score": 0.4144144144144144}, {"class": "all", "map@50:95": 0.09852571258484757, "map@50": 0.20257629765823862, "precision": 0.21757577383878005, "recall": 0.27972371996327383, "f1_score": 0.20908523978550062}], "map": 0.20257629765823862, "precision": 0.21757577383878005, "recall": 0.27972371996327383, "f1_score": 0.20908523978550062}, "test_coco_eval_bbox": [0.09852571258484757, 0.20257629765823862, 0.0857984137100992, 0.062673619841446, 0.1260643620655681, 0.1723511348326477, 0.07363581687959823, 0.19318844393057583, 0.24393604480901213, 0.1822742996013266, 0.2974083458075185, 0.3663403118526047], "epoch": 2, "n_parameters": 126141694, "best_res": 0.09852571258484757, "best_ep": 2, "now_time": "2026-02-01 08:43:08.664449", "train_epoch_time": "1:10:55", "epoch_time": "1:17:38"}
{"train_lr": 9.999999999999097e-05, "train_class_error": 18.495296075778047, "train_loss": 10.72854609409175, "train_loss_ce": 0.7613282277566388, "train_loss_bbox": 0.1658513231505972, "train_loss_giou": 0.8433611050769525, "train_loss_ce_0": 0.7659882177632655, "train_loss_bbox_0": 0.17209892585767347, "train_loss_giou_0": 0.8626522280677437, "train_loss_ce_1": 0.7595224624401976, "train_loss_bbox_1": 0.16949760194662764, "train_loss_giou_1": 0.8565113379068245, "train_loss_ce_2": 0.7606745930791816, "train_loss_bbox_2": 0.16760283187897335, "train_loss_giou_2": 0.8497545823215329, "train_loss_ce_3": 0.7620536696499485, "train_loss_bbox_3": 0.16604799958603744, "train_loss_giou_3": 0.8447220170290063, "train_loss_ce_enc": 0.7518084228999432, "train_loss_bbox_enc": 0.18063021917154992, "train_loss_giou_enc": 0.8884403157429056, "train_loss_ce_unscaled": 0.7613282277566388, "train_class_error_unscaled": 18.495296075778047, "train_loss_bbox_unscaled": 0.033170264648084036, "train_loss_giou_unscaled": 0.42168055253847625, "train_cardinality_error_unscaled": 3832.811578438482, "train_loss_ce_0_unscaled": 0.7659882177632655, "train_loss_bbox_0_unscaled": 0.034419785189516175, "train_loss_giou_0_unscaled": 0.43132611403387183, "train_cardinality_error_0_unscaled": 3833.2495769881557, "train_loss_ce_1_unscaled": 0.7595224624401976, "train_loss_bbox_1_unscaled": 0.033899520396442145, "train_loss_giou_1_unscaled": 0.42825566895341227, "train_cardinality_error_1_unscaled": 3833.1273869954075, "train_loss_ce_2_unscaled": 0.7606745930791816, "train_loss_bbox_2_unscaled": 0.03352056637619707, "train_loss_giou_2_unscaled": 0.42487729116076645, "train_cardinality_error_2_unscaled": 3833.040246555475, "train_loss_ce_3_unscaled": 0.7620536696499485, "train_loss_bbox_3_unscaled": 0.033209599937215055, "train_loss_giou_3_unscaled": 0.42236100851450314, "train_cardinality_error_3_unscaled": 3832.4328015470146, "train_loss_ce_enc_unscaled": 0.7518084228999432, "train_loss_bbox_enc_unscaled": 0.036126043852021325, "train_loss_giou_enc_unscaled": 0.4442201578714528, "train_cardinality_error_enc_unscaled": 3832.957215373459, "test_class_error": 19.438856736440506, "test_loss": 10.833167027367486, "test_loss_ce": 0.8200403025793651, "test_loss_bbox": 0.1494260047861035, "test_loss_giou": 0.8148248208326007, "test_loss_ce_0": 0.8191524057539683, "test_loss_bbox_0": 0.15419226285957155, "test_loss_giou_0": 0.8432952370340862, "test_loss_ce_1": 0.8149503968253968, "test_loss_bbox_1": 0.15342761358689694, "test_loss_giou_1": 0.8340716758039263, "test_loss_ce_2": 0.8201884920634921, "test_loss_bbox_2": 0.15182847607703437, "test_loss_giou_2": 0.8247980776476481, "test_loss_ce_3": 0.8188306051587302, "test_loss_bbox_3": 0.15089659590155832, "test_loss_giou_3": 0.8168867682369928, "test_loss_ce_enc": 0.8064918154761904, "test_loss_bbox_enc": 0.1626605396020034, "test_loss_giou_enc": 0.8772049325136911, "test_loss_ce_unscaled": 0.8200403025793651, "test_class_error_unscaled": 19.438856736440506, "test_loss_bbox_unscaled": 0.029885200976083676, "test_loss_giou_unscaled": 0.40741241041630033, "test_cardinality_error_unscaled": 236.65396825396826, "test_loss_ce_0_unscaled": 0.8191524057539683, "test_loss_bbox_0_unscaled": 0.03083845256502548, "test_loss_giou_0_unscaled": 0.4216476185170431, "test_cardinality_error_0_unscaled": 236.68571428571428, "test_loss_ce_1_unscaled": 0.8149503968253968, "test_loss_bbox_1_unscaled": 0.03068552268846404, "test_loss_giou_1_unscaled": 0.41703583790196314, "test_cardinality_error_1_unscaled": 236.67873015873016, "test_loss_ce_2_unscaled": 0.8201884920634921, "test_loss_bbox_2_unscaled": 0.030365695239502995, "test_loss_giou_2_unscaled": 0.41239903882382406, "test_cardinality_error_2_unscaled": 236.66603174603173, "test_loss_ce_3_unscaled": 0.8188306051587302, "test_loss_bbox_3_unscaled": 0.030179319181553428, "test_loss_giou_3_unscaled": 0.4084433841184964, "test_cardinality_error_3_unscaled": 236.68, "test_loss_ce_enc_unscaled": 0.8064918154761904, "test_loss_bbox_enc_unscaled": 0.032532107872031035, "test_loss_giou_enc_unscaled": 0.43860246625684557, "test_cardinality_error_enc_unscaled": 236.69333333333333, "test_results_json": {"class_map": [{"class": "Aircraft Hangar", "map@50:95": 0.05958738681191571, "map@50": 0.0725117488707396, "precision": 1.0, "recall": 0.041666666666666664, "f1_score": 0.07999999999999999}, {"class": "Barge", "map@50:95": 0.0652575738269702, "map@50": 0.10462558090950595, "precision": 0.3333333333333333, "recall": 0.045454545454545456, "f1_score": 0.08}, {"class": "Building", "map@50:95": 0.24833076913344243, "map@50": 0.5655417514481492, "precision": 0.5645775108521139, "recall": 0.642551391591236, "f1_score": 0.6010461095328908}, {"class": "Bus", "map@50:95": 0.09331897432869163, "map@50": 0.23388249350934445, "precision": 0.447242206235012, "recall": 0.26360424028268553, "f1_score": 0.331702979101823}, {"class": "Cargo Car", "map@50:95": 0.2982471229968732, "map@50": 0.582390475907271, "precision": 0.4016393442622951, "recall": 0.7424242424242424, "f1_score": 0.5212765957446809}, {"class": "Cargo Plane", "map@50:95": 0.5018871718396942, "map@50": 0.8120785590881887, "precision": 0.7575757575757576, "recall": 0.8426966292134831, "f1_score": 0.7978723404255319}, {"class": "Cargo Truck", "map@50:95": 0.015367930045013875, "map@50": 0.04876280015749044, "precision": 0.21666666666666667, "recall": 0.04486626402070751, "f1_score": 0.07433881343817013}, {"class": "Cement Mixer", "map@50:95": 0.07443202748556019, "map@50": 0.23746825425976484, "precision": 0.625, "recall": 0.18181818181818182, "f1_score": 0.28169014084507044}, {"class": "Construction Site", "map@50:95": 0.002135022417063039, "map@50": 0.006363213764293566, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Container Crane", "map@50:95": 0.05907390334175118, "map@50": 0.09582537201088528, "precision": 0.5, "recall": 0.05263157894736842, "f1_score": 0.09523809523809525}, {"class": "Container Ship", "map@50:95": 0.3715459429016167, "map@50": 0.5247972306255873, "precision": 0.6222222222222222, "recall": 0.5384615384615384, "f1_score": 0.577319587628866}, {"class": "Crane Truck", "map@50:95": 0.004047029581638202, "map@50": 0.009216895484436828, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Damaged Building", "map@50:95": 0.05329297742417839, "map@50": 0.10921988924215713, "precision": 0.3053435114503817, "recall": 0.14336917562724014, "f1_score": 0.1951219512195122}, {"class": "Dump Truck", "map@50:95": 0.06299065692130995, "map@50": 0.13057810361885588, "precision": 0.4444444444444444, "recall": 0.053691275167785234, "f1_score": 0.09580838323353294}, {"class": "Engineering Vehicle", "map@50:95": 0.04563650853654983, "map@50": 0.09119028006498592, "precision": 0.6666666666666666, "recall": 0.044444444444444446, "f1_score": 0.08333333333333334}, {"class": "Excavator", "map@50:95": 0.14797569231134344, "map@50": 0.31337050456509624, "precision": 0.40714285714285714, "recall": 0.3131868131868132, "f1_score": 0.3540372670807454}, {"class": "Facility", "map@50:95": 0.10447211284633517, "map@50": 0.1728839860717401, "precision": 0.36363636363636365, "recall": 0.22018348623853212, "f1_score": 0.2742857142857143}, {"class": "Ferry", "map@50:95": 0.026704767911713462, "map@50": 0.06317882114679226, "precision": 0.16666666666666666, "recall": 0.08333333333333333, "f1_score": 0.1111111111111111}, {"class": "Fishing Vessel", "map@50:95": 0.054482079450118105, "map@50": 0.11159525440687552, "precision": 0.3333333333333333, "recall": 0.025974025974025976, "f1_score": 0.04819277108433735}, {"class": "Fixed-wing Aircraft", "map@50:95": 0.04593144797987887, "map@50": 0.09685320116563344, "precision": 0.18181818181818182, "recall": 0.09090909090909091, "f1_score": 0.12121212121212123}, {"class": "Flat Car", "map@50:95": 0.0032844764139808085, "map@50": 0.007345895879910571, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Front loader-Bulldozer", "map@50:95": 0.056076551429087976, "map@50": 0.1517723881104496, "precision": 0.3614457831325301, "recall": 0.21428571428571427, "f1_score": 0.26905829596412556}, {"class": "Ground Grader", "map@50:95": 0.033390075849690226, "map@50": 0.051925192519251914, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Haul Truck", "map@50:95": 0.5277370930132601, "map@50": 0.9191079918914735, "precision": 0.8372093023255814, "recall": 0.8571428571428571, "f1_score": 0.8470588235294119}, {"class": "Helicopter", "map@50:95": 0.06204417364813404, "map@50": 0.2739070830159939, "precision": 0.2727272727272727, "recall": 0.25, "f1_score": 0.2608695652173913}, {"class": "Helipad", "map@50:95": 0.1104366203345267, "map@50": 0.18101458962041128, "precision": 0.3333333333333333, "recall": 0.08333333333333333, "f1_score": 0.13333333333333333}, {"class": "Hut-Tent", "map@50:95": 0.0003482106805913169, "map@50": 0.0010678215358951666, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Locomotive", "map@50:95": 0.03470452584177706, "map@50": 0.09405334087670236, "precision": 0.25, "recall": 0.05263157894736842, "f1_score": 0.08695652173913043}, {"class": "Maritime Vessel", "map@50:95": 0.12232872750271558, "map@50": 0.22413465512865868, "precision": 0.3, "recall": 0.3, "f1_score": 0.3}, {"class": "Mobile Crane", "map@50:95": 0.040049934297031214, "map@50": 0.08807995727744283, "precision": 0.375, "recall": 0.058823529411764705, "f1_score": 0.10169491525423728}, {"class": "Motorboat", "map@50:95": 0.10356753256903425, "map@50": 0.297478604168072, "precision": 0.3453815261044177, "recall": 0.5180722891566265, "f1_score": 0.4144578313253012}, {"class": "Oil Tanker", "map@50:95": 0.22231825481315742, "map@50": 0.34164498891562883, "precision": 0.2857142857142857, "recall": 0.3333333333333333, "f1_score": 0.30769230769230765}, {"class": "Passenger Car", "map@50:95": 0.32414397058747335, "map@50": 0.7173632892649026, "precision": 0.7473684210526316, "recall": 0.7421602787456446, "f1_score": 0.7447552447552447}, {"class": "Passenger Vehicle", "map@50:95": 0.0005435926942344307, "map@50": 0.0019413353852209977, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Pickup Truck", "map@50:95": 0.0013353622490130715, "map@50": 0.00319585262111514, "precision": 0.125, "recall": 0.0056179775280898875, "f1_score": 0.01075268817204301}, {"class": "Pylon", "map@50:95": 0.2810906869051688, "map@50": 0.4853962515984462, "precision": 0.576271186440678, "recall": 0.5, "f1_score": 0.5354330708661418}, {"class": "Reach Stacker", "map@50:95": 0.0, "map@50": 0.0, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Sailboat", "map@50:95": 0.2407794755296176, "map@50": 0.47046179693235635, "precision": 0.344, "recall": 0.589041095890411, "f1_score": 0.4343434343434343}, {"class": "Scraper-Tractor", "map@50:95": 0.00826740790070771, "map@50": 0.017396386172591177, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Shed", "map@50:95": 0.09447974195900956, "map@50": 0.16189590874296034, "precision": 0.4943820224719101, "recall": 0.14814814814814814, "f1_score": 0.22797927461139894}, {"class": "Shipping Container", "map@50:95": 0.038659298399079764, "map@50": 0.09584036506278083, "precision": 0.11830985915492957, "recall": 0.36681222707423583, "f1_score": 0.17891373801916932}, {"class": "Shipping container lot", "map@50:95": 0.12019092140729706, "map@50": 0.2500310104069242, "precision": 0.2694805194805195, "recall": 0.4797687861271676, "f1_score": 0.34511434511434513}, {"class": "Small Aircraft", "map@50:95": 0.30991826053546906, "map@50": 0.6423640002846944, "precision": 0.5058823529411764, "recall": 0.6825396825396826, "f1_score": 0.581081081081081}, {"class": "Small Car", "map@50:95": 0.08477201284384248, "map@50": 0.306311482294465, "precision": 0.5243229598644117, "recall": 0.3695889784194606, "f1_score": 0.4335639312977099}, {"class": "Storage Tank", "map@50:95": 0.2964917091031594, "map@50": 0.5325550631964273, "precision": 0.6374045801526718, "recall": 0.540453074433657, "f1_score": 0.5849387040280212}, {"class": "Straddle Carrier", "map@50:95": 0.0002733402174573285, "map@50": 0.0009111340581910951, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tank car", "map@50:95": 0.3585517555838866, "map@50": 0.7405737421984219, "precision": 0.5161290322580645, "recall": 0.7272727272727273, "f1_score": 0.6037735849056604}, {"class": "Tower", "map@50:95": 0.0, "map@50": 0.0, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tower crane", "map@50:95": 0.0033172187154749586, "map@50": 0.010288022405652077, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Trailer", "map@50:95": 0.0393430707124713, "map@50": 0.10879552113026124, "precision": 0.20722891566265061, "recall": 0.23626373626373626, "f1_score": 0.220795892169448}, {"class": "Truck", "map@50:95": 0.03578031663038707, "map@50": 0.09116566565024953, "precision": 0.19003260363297625, "recall": 0.1712846347607053, "f1_score": 0.18017222344888498}, {"class": "Truck Tractor", "map@50:95": 0.00035535594653461134, "map@50": 0.001474752778772666, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Truck w-Box", "map@50:95": 0.11401754599426557, "map@50": 0.28793419691237887, "precision": 0.2635090060040027, "recall": 0.5163398692810458, "f1_score": 0.3489399293286219}, {"class": "Truck w-Flatbed", "map@50:95": 0.027237554173792738, "map@50": 0.07039350702912822, "precision": 0.25806451612903225, "recall": 0.05442176870748299, "f1_score": 0.0898876404494382}, {"class": "Truck w-Liquid", "map@50:95": 0.06653711284325818, "map@50": 0.09470069651743135, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tugboat", "map@50:95": 0.11188107416476104, "map@50": 0.27961016339309447, "precision": 0.34285714285714286, "recall": 0.375, "f1_score": 0.3582089552238806}, {"class": "Utility Truck", "map@50:95": 0.006295440580918452, "map@50": 0.018785908894481413, "precision": 0.08426966292134831, "recall": 0.01935483870967742, "f1_score": 0.03147953830010493}, {"class": "Vehicle Lot", "map@50:95": 0.014233000944250992, "map@50": 0.03765537239572651, "precision": 0.23333333333333334, "recall": 0.03436988543371522, "f1_score": 0.05991440798858773}, {"class": "Yacht", "map@50:95": 0.3634370292317877, "map@50": 0.6514632012602161, "precision": 0.3826086956521739, "recall": 0.8627450980392157, "f1_score": 0.5301204819277109}, {"class": "all", "map@50:95": 0.11174467000572817, "map@50": 0.22190460257363687, "precision": 0.3138741589771757, "recall": 0.245085972317758, "f1_score": 0.23635384872206272}], "map": 0.22190460257363687, "precision": 0.3138741589771757, "recall": 0.245085972317758, "f1_score": 0.23635384872206272}, "test_coco_eval_bbox": [0.11174467000572817, 0.22190460257363687, 0.10096673659273991, 0.07508394517663484, 0.1429429129232347, 0.19038867370694162, 0.07434948058589975, 0.19011448763856237, 0.24521400509845592, 0.1820525607824432, 0.29135858948873977, 0.31654055435990963], "epoch": 3, "n_parameters": 126141694, "best_res": 0.11174467000572817, "best_ep": 3, "now_time": "2026-02-01 10:01:03.634966", "train_epoch_time": "1:11:12", "epoch_time": "1:17:54"}
{"train_lr": 9.999999999999097e-05, "train_class_error": 16.944897505290157, "train_loss": 10.571509451745685, "train_loss_ce": 0.7449144983942967, "train_loss_bbox": 0.1643827436536871, "train_loss_giou": 0.8351253634735182, "train_loss_ce_0": 0.7504614740584233, "train_loss_bbox_0": 0.17032038072031375, "train_loss_giou_0": 0.853584448729275, "train_loss_ce_1": 0.7433168077296328, "train_loss_bbox_1": 0.16790557797711836, "train_loss_giou_1": 0.8474837126944674, "train_loss_ce_2": 0.7437703119062578, "train_loss_bbox_2": 0.16589117232079337, "train_loss_giou_2": 0.8412202347073581, "train_loss_ce_3": 0.7445440340735902, "train_loss_bbox_3": 0.16473877008007973, "train_loss_giou_3": 0.8369075269040989, "train_loss_ce_enc": 0.7376662695815909, "train_loss_bbox_enc": 0.1789006325698793, "train_loss_giou_enc": 0.880375494887381, "train_loss_ce_unscaled": 0.7449144983942967, "train_class_error_unscaled": 16.944897505290157, "train_loss_bbox_unscaled": 0.03287654875242213, "train_loss_giou_unscaled": 0.4175626817367591, "train_cardinality_error_unscaled": 3833.186004350979, "train_loss_ce_0_unscaled": 0.7504614740584233, "train_loss_bbox_0_unscaled": 0.03406407615298032, "train_loss_giou_0_unscaled": 0.4267922243646375, "train_cardinality_error_0_unscaled": 3833.383732173072, "train_loss_ce_1_unscaled": 0.7433168077296328, "train_loss_bbox_1_unscaled": 0.03358111559301489, "train_loss_giou_1_unscaled": 0.4237418563472337, "train_cardinality_error_1_unscaled": 3833.1428571428573, "train_loss_ce_2_unscaled": 0.7437703119062578, "train_loss_bbox_2_unscaled": 0.033178234456043086, "train_loss_giou_2_unscaled": 0.42061011735367904, "train_cardinality_error_2_unscaled": 3833.30094271211, "train_loss_ce_3_unscaled": 0.7445440340735902, "train_loss_bbox_3_unscaled": 0.03294775401409398, "train_loss_giou_3_unscaled": 0.41845376345204943, "train_cardinality_error_3_unscaled": 3833.3335750543874, "train_loss_ce_enc_unscaled": 0.7376662695815909, "train_loss_bbox_enc_unscaled": 0.03578012655114603, "train_loss_giou_enc_unscaled": 0.4401877474436905, "train_cardinality_error_enc_unscaled": 3832.862218999275, "test_class_error": 18.978539511059957, "test_loss": 10.708728780443707, "test_loss_ce": 0.8111687748015873, "test_loss_bbox": 0.14936805829997102, "test_loss_giou": 0.8094734287735016, "test_loss_ce_0": 0.8087918526785715, "test_loss_bbox_0": 0.1560954320631803, "test_loss_giou_0": 0.8303631146748861, "test_loss_ce_1": 0.8052867683531746, "test_loss_bbox_1": 0.15252058488035958, "test_loss_giou_1": 0.8235530036120188, "test_loss_ce_2": 0.8066858878968254, "test_loss_bbox_2": 0.15132482081296897, "test_loss_giou_2": 0.8158125810679935, "test_loss_ce_3": 0.8079693700396825, "test_loss_bbox_3": 0.15010039092056335, "test_loss_giou_3": 0.8120997220277786, "test_loss_ce_enc": 0.7984204489087302, "test_loss_bbox_enc": 0.16223649858600564, "test_loss_giou_enc": 0.8574580570818886, "test_loss_ce_unscaled": 0.8111687748015873, "test_class_error_unscaled": 18.978539511059957, "test_loss_bbox_unscaled": 0.029873611683853798, "test_loss_giou_unscaled": 0.4047367143867508, "test_cardinality_error_unscaled": 236.70857142857142, "test_loss_ce_0_unscaled": 0.8087918526785715, "test_loss_bbox_0_unscaled": 0.03121908644894286, "test_loss_giou_0_unscaled": 0.41518155733744305, "test_cardinality_error_0_unscaled": 236.7174603174603, "test_loss_ce_1_unscaled": 0.8052867683531746, "test_loss_bbox_1_unscaled": 0.030504116975362337, "test_loss_giou_1_unscaled": 0.4117765018060094, "test_cardinality_error_1_unscaled": 236.70984126984126, "test_loss_ce_2_unscaled": 0.8066858878968254, "test_loss_bbox_2_unscaled": 0.030264964132968867, "test_loss_giou_2_unscaled": 0.40790629053399674, "test_cardinality_error_2_unscaled": 236.7073015873016, "test_loss_ce_3_unscaled": 0.8079693700396825, "test_loss_bbox_3_unscaled": 0.030020078222696034, "test_loss_giou_3_unscaled": 0.4060498610138893, "test_cardinality_error_3_unscaled": 236.68063492063493, "test_loss_ce_enc_unscaled": 0.7984204489087302, "test_loss_bbox_enc_unscaled": 0.03244729968231349, "test_loss_giou_enc_unscaled": 0.4287290285409443, "test_cardinality_error_enc_unscaled": 236.68761904761905, "test_results_json": {"class_map": [{"class": "Aircraft Hangar", "map@50:95": 0.016847689507701852, "map@50": 0.02108679869592744, "precision": 0.14285714285714285, "recall": 0.041666666666666664, "f1_score": 0.06451612903225806}, {"class": "Barge", "map@50:95": 0.08196167097890088, "map@50": 0.12245465288082547, "precision": 0.14444444444444443, "recall": 0.29545454545454547, "f1_score": 0.19402985074626863}, {"class": "Building", "map@50:95": 0.24313756754910093, "map@50": 0.5667684312516763, "precision": 0.4772144383756827, "recall": 0.6799932323830471, "f1_score": 0.5608372579801152}, {"class": "Bus", "map@50:95": 0.10113992440081561, "map@50": 0.25533746529008405, "precision": 0.2752212389380531, "recall": 0.43957597173144874, "f1_score": 0.3385034013605442}, {"class": "Cargo Car", "map@50:95": 0.29785356411219277, "map@50": 0.630768729540184, "precision": 0.5476190476190477, "recall": 0.696969696969697, "f1_score": 0.6133333333333334}, {"class": "Cargo Plane", "map@50:95": 0.44611316945767115, "map@50": 0.8346343098905752, "precision": 0.7037037037037037, "recall": 0.8539325842696629, "f1_score": 0.7715736040609137}, {"class": "Cargo Truck", "map@50:95": 0.016962873690903615, "map@50": 0.05343808928084254, "precision": 0.1019402525408069, "recall": 0.28559102674719583, "f1_score": 0.15024965955515204}, {"class": "Cement Mixer", "map@50:95": 0.04072478049172158, "map@50": 0.1748590472125419, "precision": 0.8571428571428571, "recall": 0.10909090909090909, "f1_score": 0.19354838709677416}, {"class": "Construction Site", "map@50:95": 0.0029270547543883128, "map@50": 0.01152405964753226, "precision": 0.07692307692307693, "recall": 0.007407407407407408, "f1_score": 0.013513513513513514}, {"class": "Container Crane", "map@50:95": 0.04991471836259257, "map@50": 0.05826191862883767, "precision": 0.3333333333333333, "recall": 0.05263157894736842, "f1_score": 0.09090909090909091}, {"class": "Container Ship", "map@50:95": 0.366275235436356, "map@50": 0.5350874297782607, "precision": 0.4146341463414634, "recall": 0.6538461538461539, "f1_score": 0.5074626865671641}, {"class": "Crane Truck", "map@50:95": 0.011256909137720659, "map@50": 0.030626405666917506, "precision": 0.09302325581395349, "recall": 0.17391304347826086, "f1_score": 0.12121212121212122}, {"class": "Damaged Building", "map@50:95": 0.05405913875502257, "map@50": 0.118998399164385, "precision": 0.1784776902887139, "recall": 0.24372759856630824, "f1_score": 0.20606060606060606}, {"class": "Dump Truck", "map@50:95": 0.07617329475741288, "map@50": 0.18342803060590324, "precision": 0.23821989528795812, "recall": 0.3053691275167785, "f1_score": 0.26764705882352946}, {"class": "Engineering Vehicle", "map@50:95": 0.04147061892066214, "map@50": 0.08236755353249388, "precision": 0.14285714285714285, "recall": 0.1111111111111111, "f1_score": 0.125}, {"class": "Excavator", "map@50:95": 0.14274107273225595, "map@50": 0.3237720442145013, "precision": 0.14330708661417324, "recall": 0.5, "f1_score": 0.22276621787025705}, {"class": "Facility", "map@50:95": 0.08783109970292198, "map@50": 0.15778903215727225, "precision": 0.24742268041237114, "recall": 0.22018348623853212, "f1_score": 0.2330097087378641}, {"class": "Ferry", "map@50:95": 0.16011037686255347, "map@50": 0.2557612262984556, "precision": 0.3333333333333333, "recall": 0.25, "f1_score": 0.28571428571428575}, {"class": "Fishing Vessel", "map@50:95": 0.044816226268907085, "map@50": 0.12644403015238806, "precision": 0.14835164835164835, "recall": 0.35064935064935066, "f1_score": 0.2084942084942085}, {"class": "Fixed-wing Aircraft", "map@50:95": 0.13926188946121293, "map@50": 0.32077334272414926, "precision": 0.45454545454545453, "recall": 0.22727272727272727, "f1_score": 0.30303030303030304}, {"class": "Flat Car", "map@50:95": 0.00794768780783002, "map@50": 0.021953638487787655, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Front loader-Bulldozer", "map@50:95": 0.07282042388484714, "map@50": 0.16328373326929194, "precision": 0.1505016722408027, "recall": 0.32142857142857145, "f1_score": 0.2050113895216401}, {"class": "Ground Grader", "map@50:95": 0.03797902726199213, "map@50": 0.062116788014083, "precision": 0.1111111111111111, "recall": 0.2, "f1_score": 0.14285714285714285}, {"class": "Haul Truck", "map@50:95": 0.4737757065309558, "map@50": 0.9288943845152822, "precision": 0.7931034482758621, "recall": 0.9126984126984127, "f1_score": 0.8487084870848709}, {"class": "Helicopter", "map@50:95": 0.04268914233195471, "map@50": 0.19047917449972843, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Helipad", "map@50:95": 0.10335572182908445, "map@50": 0.19161324489141937, "precision": 0.38461538461538464, "recall": 0.20833333333333334, "f1_score": 0.2702702702702703}, {"class": "Hut-Tent", "map@50:95": 9.281944827419026e-05, "map@50": 0.0004140752326402125, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Locomotive", "map@50:95": 0.03416979770129193, "map@50": 0.07325696146906072, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Maritime Vessel", "map@50:95": 0.09591364680614202, "map@50": 0.1748351615051828, "precision": 0.16, "recall": 0.45, "f1_score": 0.23606557377049184}, {"class": "Mobile Crane", "map@50:95": 0.027963909305094965, "map@50": 0.07067191762933027, "precision": 0.13580246913580246, "recall": 0.21568627450980393, "f1_score": 0.16666666666666666}, {"class": "Motorboat", "map@50:95": 0.1361024607879022, "map@50": 0.3670638305411047, "precision": 0.4968944099378882, "recall": 0.4819277108433735, "f1_score": 0.4892966360856269}, {"class": "Oil Tanker", "map@50:95": 0.15533623930957313, "map@50": 0.3433256877805952, "precision": 0.14285714285714285, "recall": 0.3333333333333333, "f1_score": 0.2}, {"class": "Passenger Car", "map@50:95": 0.3152742276172848, "map@50": 0.7357440900062182, "precision": 0.7551724137931034, "recall": 0.7630662020905923, "f1_score": 0.7590987868284228}, {"class": "Passenger Vehicle", "map@50:95": 0.003016783538924333, "map@50": 0.00817507221681996, "precision": 0.022556390977443608, "recall": 0.011278195488721804, "f1_score": 0.015037593984962405}, {"class": "Pickup Truck", "map@50:95": 0.0016018460459219415, "map@50": 0.005631538040867933, "precision": 0.08, "recall": 0.011235955056179775, "f1_score": 0.019704433497536946}, {"class": "Pylon", "map@50:95": 0.2529927345729142, "map@50": 0.4920236080115587, "precision": 0.5633802816901409, "recall": 0.5882352941176471, "f1_score": 0.5755395683453237}, {"class": "Reach Stacker", "map@50:95": 0.03877887788778878, "map@50": 0.05610561056105611, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Sailboat", "map@50:95": 0.2056859590410693, "map@50": 0.4628507611404182, "precision": 0.28651685393258425, "recall": 0.6986301369863014, "f1_score": 0.4063745019920318}, {"class": "Scraper-Tractor", "map@50:95": 0.05354706668739423, "map@50": 0.13161971704287717, "precision": 0.2222222222222222, "recall": 0.18181818181818182, "f1_score": 0.19999999999999998}, {"class": "Shed", "map@50:95": 0.06709924126681907, "map@50": 0.15740299646529982, "precision": 0.34355828220858897, "recall": 0.18855218855218855, "f1_score": 0.2434782608695652}, {"class": "Shipping Container", "map@50:95": 0.06549348217432006, "map@50": 0.17272659100973928, "precision": 0.1303030303030303, "recall": 0.37554585152838427, "f1_score": 0.1934758155230596}, {"class": "Shipping container lot", "map@50:95": 0.1431967413144632, "map@50": 0.27659537871448486, "precision": 0.2026431718061674, "recall": 0.5317919075144508, "f1_score": 0.29346092503987237}, {"class": "Small Aircraft", "map@50:95": 0.2022707955742391, "map@50": 0.563318733908391, "precision": 0.4895833333333333, "recall": 0.746031746031746, "f1_score": 0.5911949685534592}, {"class": "Small Car", "map@50:95": 0.08875336689296949, "map@50": 0.3202308839733519, "precision": 0.4090856468082151, "recall": 0.4490989044508274, "f1_score": 0.42815945716709075}, {"class": "Storage Tank", "map@50:95": 0.28704322083883715, "map@50": 0.5438877683249342, "precision": 0.556923076923077, "recall": 0.5857605177993528, "f1_score": 0.5709779179810726}, {"class": "Straddle Carrier", "map@50:95": 0.004198618213469698, "map@50": 0.010296485692525295, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tank car", "map@50:95": 0.1723473997112228, "map@50": 0.5866991298864517, "precision": 0.6521739130434783, "recall": 0.6818181818181818, "f1_score": 0.6666666666666666}, {"class": "Tower", "map@50:95": 0.018089002448631958, "map@50": 0.045065474289364416, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tower crane", "map@50:95": 0.012406067529829905, "map@50": 0.02690544695495191, "precision": 0.1111111111111111, "recall": 0.03571428571428571, "f1_score": 0.05405405405405406}, {"class": "Trailer", "map@50:95": 0.037911389199399834, "map@50": 0.09833928549620677, "precision": 0.16902654867256636, "recall": 0.2623626373626374, "f1_score": 0.20559741657696448}, {"class": "Truck", "map@50:95": 0.03327909770037944, "map@50": 0.08346859046888977, "precision": 0.1343159642931285, "recall": 0.2716204869857263, "f1_score": 0.17974718710932072}, {"class": "Truck Tractor", "map@50:95": 0.004433018695745755, "map@50": 0.014063452394687322, "precision": 0.1346153846153846, "recall": 0.041916167664670656, "f1_score": 0.0639269406392694}, {"class": "Truck w-Box", "map@50:95": 0.11603042791815454, "map@50": 0.291244002779405, "precision": 0.23919146546883774, "recall": 0.5568627450980392, "f1_score": 0.33464257659073054}, {"class": "Truck w-Flatbed", "map@50:95": 0.02641023547255465, "map@50": 0.08356205845077685, "precision": 0.11469534050179211, "recall": 0.21768707482993196, "f1_score": 0.15023474178403756}, {"class": "Truck w-Liquid", "map@50:95": 0.00611970125583987, "map@50": 0.015648171960053147, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tugboat", "map@50:95": 0.07299965057668703, "map@50": 0.14421818441440332, "precision": 0.16666666666666666, "recall": 0.5, "f1_score": 0.25}, {"class": "Utility Truck", "map@50:95": 0.007052805534453766, "map@50": 0.02422148517799658, "precision": 0.12547528517110265, "recall": 0.04258064516129032, "f1_score": 0.06358381502890174}, {"class": "Vehicle Lot", "map@50:95": 0.01221472460518105, "map@50": 0.030286858645404955, "precision": 0.1392857142857143, "recall": 0.06382978723404255, "f1_score": 0.08754208754208755}, {"class": "Yacht", "map@50:95": 0.2799121880552857, "map@50": 0.5749749502728778, "precision": 0.32142857142857145, "recall": 0.7058823529411765, "f1_score": 0.44171779141104295}, {"class": "all", "map@50:95": 0.10403193438501253, "map@50": 0.22724399865676737, "precision": 0.2508370877483833, "recall": 0.30734090340234843, "f1_score": 0.251262764365093}], "map": 0.22724399865676737, "precision": 0.2508370877483833, "recall": 0.30734090340234843, "f1_score": 0.251262764365093}, "test_coco_eval_bbox": [0.10403193438501253, 0.22724399865676737, 0.07872076956803108, 0.07620201883012784, 0.1358401919435653, 0.17147246876817288, 0.07640742516054812, 0.19781562949278791, 0.2529974842328794, 0.19132909458065275, 0.30179727718515104, 0.3114907045843291], "epoch": 4, "n_parameters": 126141694, "best_res": 0.11174467000572817, "best_ep": 3, "now_time": "2026-02-01 11:18:44.171050", "train_epoch_time": "1:11:06", "epoch_time": "1:17:40"}
{"train_lr": 9.999999999999097e-05, "train_class_error": 15.239420015050618, "train_loss": 10.34945450421496, "train_loss_ce": 0.7313844380580142, "train_loss_bbox": 0.15890960684244254, "train_loss_giou": 0.816135897047427, "train_loss_ce_0": 0.7379961778941738, "train_loss_bbox_0": 0.16500358010752764, "train_loss_giou_0": 0.8348125176527217, "train_loss_ce_1": 0.7307325264809648, "train_loss_bbox_1": 0.16259727420485476, "train_loss_giou_1": 0.8283367555719954, "train_loss_ce_2": 0.7309232139888174, "train_loss_bbox_2": 0.16074342861723634, "train_loss_giou_2": 0.8213512842746706, "train_loss_ce_3": 0.7309784408701403, "train_loss_bbox_3": 0.15937973999947125, "train_loss_giou_3": 0.8181017021476819, "train_loss_ce_enc": 0.727558304131897, "train_loss_bbox_enc": 0.1733827803692298, "train_loss_giou_enc": 0.8611268416319041, "train_loss_ce_unscaled": 0.7313844380580142, "train_class_error_unscaled": 15.239420015050618, "train_loss_bbox_unscaled": 0.03178192140320769, "train_loss_giou_unscaled": 0.4080679485237135, "train_cardinality_error_unscaled": 3832.967125936669, "train_loss_ce_0_unscaled": 0.7379961778941738, "train_loss_bbox_0_unscaled": 0.03300071604102908, "train_loss_giou_0_unscaled": 0.41740625882636084, "train_cardinality_error_0_unscaled": 3833.11022480058, "train_loss_ce_1_unscaled": 0.7307325264809648, "train_loss_bbox_1_unscaled": 0.032519454850501985, "train_loss_giou_1_unscaled": 0.4141683777859977, "train_cardinality_error_1_unscaled": 3832.9633792603336, "train_loss_ce_2_unscaled": 0.7309232139888174, "train_loss_bbox_2_unscaled": 0.03214868575871799, "train_loss_giou_2_unscaled": 0.4106756421373353, "train_cardinality_error_2_unscaled": 3832.710780759004, "train_loss_ce_3_unscaled": 0.7309784408701403, "train_loss_bbox_3_unscaled": 0.031875948021154045, "train_loss_giou_3_unscaled": 0.4090508510738409, "train_cardinality_error_3_unscaled": 3832.810974135847, "train_loss_ce_enc_unscaled": 0.727558304131897, "train_loss_bbox_enc_unscaled": 0.03467655607846092, "train_loss_giou_enc_unscaled": 0.43056342081595206, "train_cardinality_error_enc_unscaled": 3832.833937635968, "test_class_error": 19.600399230472625, "test_loss": 10.74111314077226, "test_loss_ce": 0.8276060267857143, "test_loss_bbox": 0.14519488993381696, "test_loss_giou": 0.8038733592203685, "test_loss_ce_0": 0.8284263392857143, "test_loss_bbox_0": 0.1505091870289355, "test_loss_giou_0": 0.8250563921531041, "test_loss_ce_1": 0.8214707341269841, "test_loss_bbox_1": 0.14781811991676924, "test_loss_giou_1": 0.8172155203611132, "test_loss_ce_2": 0.8293557787698412, "test_loss_bbox_2": 0.14579721709979432, "test_loss_giou_2": 0.8074648566567708, "test_loss_ce_3": 0.8279520089285715, "test_loss_bbox_3": 0.1447545342620403, "test_loss_giou_3": 0.8044590187924249, "test_loss_ce_enc": 0.8041834077380953, "test_loss_bbox_enc": 0.1580491502470677, "test_loss_giou_enc": 0.8519266186063251, "test_loss_ce_unscaled": 0.8276060267857143, "test_class_error_unscaled": 19.600399230472625, "test_loss_bbox_unscaled": 0.02903897797097526, "test_loss_giou_unscaled": 0.40193667961018426, "test_cardinality_error_unscaled": 236.5911111111111, "test_loss_ce_0_unscaled": 0.8284263392857143, "test_loss_bbox_0_unscaled": 0.03010183738278491, "test_loss_giou_0_unscaled": 0.41252819607655206, "test_cardinality_error_0_unscaled": 236.72126984126984, "test_loss_ce_1_unscaled": 0.8214707341269841, "test_loss_bbox_1_unscaled": 0.02956362398518693, "test_loss_giou_1_unscaled": 0.4086077601805566, "test_cardinality_error_1_unscaled": 236.6736507936508, "test_loss_ce_2_unscaled": 0.8293557787698412, "test_loss_bbox_2_unscaled": 0.029159443405057703, "test_loss_giou_2_unscaled": 0.4037324283283854, "test_cardinality_error_2_unscaled": 236.41142857142856, "test_loss_ce_3_unscaled": 0.8279520089285715, "test_loss_bbox_3_unscaled": 0.028950906873991093, "test_loss_giou_3_unscaled": 0.40222950939621244, "test_cardinality_error_3_unscaled": 236.36698412698414, "test_loss_ce_enc_unscaled": 0.8041834077380953, "test_loss_bbox_enc_unscaled": 0.03160983004769872, "test_loss_giou_enc_unscaled": 0.42596330930316256, "test_cardinality_error_enc_unscaled": 236.7295238095238, "test_results_json": {"class_map": [{"class": "Aircraft Hangar", "map@50:95": 0.013033644310486892, "map@50": 0.02517472335468841, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Barge", "map@50:95": 0.10586289245493999, "map@50": 0.19099183751479704, "precision": 0.3333333333333333, "recall": 0.1590909090909091, "f1_score": 0.21538461538461537}, {"class": "Building", "map@50:95": 0.2498316716151195, "map@50": 0.5696385288684905, "precision": 0.5913619167717529, "recall": 0.634734794010659, "f1_score": 0.6122812028234527}, {"class": "Bus", "map@50:95": 0.09094860869011669, "map@50": 0.2273348085987272, "precision": 0.40451745379876797, "recall": 0.2784452296819788, "f1_score": 0.3298451234826288}, {"class": "Cargo Car", "map@50:95": 0.3554751062842265, "map@50": 0.6295102920595882, "precision": 0.4375, "recall": 0.6893939393939394, "f1_score": 0.5352941176470588}, {"class": "Cargo Plane", "map@50:95": 0.5218584365243972, "map@50": 0.8475987574095949, "precision": 0.6209677419354839, "recall": 0.8651685393258427, "f1_score": 0.7230046948356808}, {"class": "Cargo Truck", "map@50:95": 0.017180446650244412, "map@50": 0.050169388046058, "precision": 0.17301038062283736, "recall": 0.08628127696289906, "f1_score": 0.11514104778353483}, {"class": "Cement Mixer", "map@50:95": 0.07561058091330497, "map@50": 0.21297529698587347, "precision": 0.3333333333333333, "recall": 0.18181818181818182, "f1_score": 0.23529411764705885}, {"class": "Construction Site", "map@50:95": 0.0019432971863208679, "map@50": 0.00755143580376892, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Container Crane", "map@50:95": 0.04174666605256019, "map@50": 0.07661920967043499, "precision": 0.14814814814814814, "recall": 0.21052631578947367, "f1_score": 0.17391304347826086}, {"class": "Container Ship", "map@50:95": 0.3409682034220597, "map@50": 0.5396630991640556, "precision": 0.5102040816326531, "recall": 0.4807692307692308, "f1_score": 0.4950495049504951}, {"class": "Crane Truck", "map@50:95": 0.022719253464041675, "map@50": 0.040947460463395584, "precision": 0.2, "recall": 0.08695652173913043, "f1_score": 0.12121212121212122}, {"class": "Damaged Building", "map@50:95": 0.05793980445730096, "map@50": 0.12675873631761211, "precision": 0.29, "recall": 0.2078853046594982, "f1_score": 0.24217118997912315}, {"class": "Dump Truck", "map@50:95": 0.0717816305945301, "map@50": 0.17032342119362273, "precision": 0.26838235294117646, "recall": 0.24496644295302014, "f1_score": 0.25614035087719295}, {"class": "Engineering Vehicle", "map@50:95": 0.03376508020550325, "map@50": 0.04509658402755264, "precision": 1.0, "recall": 0.022222222222222223, "f1_score": 0.04347826086956522}, {"class": "Excavator", "map@50:95": 0.14813770221367134, "map@50": 0.3113814708911949, "precision": 0.4342105263157895, "recall": 0.3626373626373626, "f1_score": 0.39520958083832336}, {"class": "Facility", "map@50:95": 0.0980732713741695, "map@50": 0.18450749863644633, "precision": 0.2625, "recall": 0.1926605504587156, "f1_score": 0.22222222222222224}, {"class": "Ferry", "map@50:95": 0.14747876580631297, "map@50": 0.3014932742877786, "precision": 0.4, "recall": 0.3333333333333333, "f1_score": 0.3636363636363636}, {"class": "Fishing Vessel", "map@50:95": 0.07428436568181136, "map@50": 0.17229258226904837, "precision": 0.4375, "recall": 0.09090909090909091, "f1_score": 0.15053763440860213}, {"class": "Fixed-wing Aircraft", "map@50:95": 0.07104667370838676, "map@50": 0.2232250264874685, "precision": 0.3333333333333333, "recall": 0.18181818181818182, "f1_score": 0.23529411764705885}, {"class": "Flat Car", "map@50:95": 0.017057045820861157, "map@50": 0.03294612019341469, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Front loader-Bulldozer", "map@50:95": 0.05396700222170793, "map@50": 0.13568422885454556, "precision": 0.3132530120481928, "recall": 0.18571428571428572, "f1_score": 0.23318385650224216}, {"class": "Ground Grader", "map@50:95": 0.025330553060829552, "map@50": 0.05401476674817667, "precision": 0.25, "recall": 0.06666666666666667, "f1_score": 0.10526315789473685}, {"class": "Haul Truck", "map@50:95": 0.5031382504314665, "map@50": 0.9144194837710389, "precision": 0.8699186991869918, "recall": 0.8492063492063492, "f1_score": 0.8594377510040161}, {"class": "Helicopter", "map@50:95": 0.10919301607580109, "map@50": 0.33865644628979025, "precision": 0.6, "recall": 0.25, "f1_score": 0.35294117647058826}, {"class": "Helipad", "map@50:95": 0.12091489918222591, "map@50": 0.24940263257094944, "precision": 0.5, "recall": 0.20833333333333334, "f1_score": 0.29411764705882354}, {"class": "Hut-Tent", "map@50:95": 0.0002902286199293513, "map@50": 0.0007516334011325346, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Locomotive", "map@50:95": 0.030688099627403896, "map@50": 0.05885384483611581, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Maritime Vessel", "map@50:95": 0.1091073653417806, "map@50": 0.2099882412848145, "precision": 0.17972350230414746, "recall": 0.4875, "f1_score": 0.2626262626262626}, {"class": "Mobile Crane", "map@50:95": 0.013891774744819136, "map@50": 0.040031350258340724, "precision": 0.14285714285714285, "recall": 0.0196078431372549, "f1_score": 0.034482758620689655}, {"class": "Motorboat", "map@50:95": 0.13240857278140164, "map@50": 0.37195360944589895, "precision": 0.45977011494252873, "recall": 0.4819277108433735, "f1_score": 0.4705882352941177}, {"class": "Oil Tanker", "map@50:95": 0.28355592177828526, "map@50": 0.3551509803667128, "precision": 0.18181818181818182, "recall": 0.3333333333333333, "f1_score": 0.23529411764705885}, {"class": "Passenger Car", "map@50:95": 0.34525621616268765, "map@50": 0.7105463727207968, "precision": 0.7453874538745388, "recall": 0.7038327526132404, "f1_score": 0.7240143369175628}, {"class": "Passenger Vehicle", "map@50:95": 0.0016717324722464726, "map@50": 0.005108316292714214, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Pickup Truck", "map@50:95": 0.0019932808288456584, "map@50": 0.004850533805439418, "precision": 0.022222222222222223, "recall": 0.0056179775280898875, "f1_score": 0.008968609865470852}, {"class": "Pylon", "map@50:95": 0.2503223147048178, "map@50": 0.48840102695507076, "precision": 0.5967741935483871, "recall": 0.5441176470588235, "f1_score": 0.5692307692307692}, {"class": "Reach Stacker", "map@50:95": 0.0007873076464272934, "map@50": 0.0013121794107121556, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Sailboat", "map@50:95": 0.24967393469687255, "map@50": 0.494658624385078, "precision": 0.3202614379084967, "recall": 0.6712328767123288, "f1_score": 0.43362831858407075}, {"class": "Scraper-Tractor", "map@50:95": 0.0023070873455792286, "map@50": 0.006865158379306565, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Shed", "map@50:95": 0.06631772589261643, "map@50": 0.1379875002439771, "precision": 0.34459459459459457, "recall": 0.1717171717171717, "f1_score": 0.22921348314606743}, {"class": "Shipping Container", "map@50:95": 0.04702616272570796, "map@50": 0.09460655727103047, "precision": 0.2727272727272727, "recall": 0.039301310043668124, "f1_score": 0.0687022900763359}, {"class": "Shipping container lot", "map@50:95": 0.1354742675756872, "map@50": 0.3081746461390872, "precision": 0.2575558475689882, "recall": 0.5664739884393064, "f1_score": 0.3541102077687444}, {"class": "Small Aircraft", "map@50:95": 0.2860608457941619, "map@50": 0.6413042779421091, "precision": 0.6101694915254238, "recall": 0.5714285714285714, "f1_score": 0.5901639344262296}, {"class": "Small Car", "map@50:95": 0.09272931888444348, "map@50": 0.32739473303044714, "precision": 0.4698448203550958, "recall": 0.42713708344983603, "f1_score": 0.4474742364125369}, {"class": "Storage Tank", "map@50:95": 0.29629232032028524, "map@50": 0.544057286363376, "precision": 0.5325443786982249, "recall": 0.5825242718446602, "f1_score": 0.5564142194744978}, {"class": "Straddle Carrier", "map@50:95": 0.06432018538781101, "map@50": 0.11278055029222595, "precision": 0.14285714285714285, "recall": 0.42857142857142855, "f1_score": 0.21428571428571427}, {"class": "Tank car", "map@50:95": 0.3046855388980193, "map@50": 0.6953942914534367, "precision": 0.8, "recall": 0.7272727272727273, "f1_score": 0.761904761904762}, {"class": "Tower", "map@50:95": 0.040663784688327986, "map@50": 0.08441988565053687, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tower crane", "map@50:95": 0.005902039673013029, "map@50": 0.015760775673030716, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Trailer", "map@50:95": 0.029240714565843772, "map@50": 0.08533177487850346, "precision": 0.24494949494949494, "recall": 0.13324175824175824, "f1_score": 0.17259786476868327}, {"class": "Truck", "map@50:95": 0.03725278215092348, "map@50": 0.09742794801619867, "precision": 0.17163289630512515, "recall": 0.24181360201511334, "f1_score": 0.20076681770651794}, {"class": "Truck Tractor", "map@50:95": 0.002856249435372813, "map@50": 0.01175610437061714, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Truck w-Box", "map@50:95": 0.11192972668290488, "map@50": 0.2653310571147857, "precision": 0.3077571669477234, "recall": 0.477124183006536, "f1_score": 0.3741670937980523}, {"class": "Truck w-Flatbed", "map@50:95": 0.03455669105172286, "map@50": 0.0848243442016999, "precision": 0.205607476635514, "recall": 0.14965986394557823, "f1_score": 0.1732283464566929}, {"class": "Truck w-Liquid", "map@50:95": 0.07939840421413484, "map@50": 0.15097874974478923, "precision": 1.0, "recall": 0.05263157894736842, "f1_score": 0.1}, {"class": "Tugboat", "map@50:95": 0.06503605807630915, "map@50": 0.17153191002324852, "precision": 0.35714285714285715, "recall": 0.15625, "f1_score": 0.21739130434782608}, {"class": "Utility Truck", "map@50:95": 0.007249972745836222, "map@50": 0.02228009417850255, "precision": 0.09090909090909091, "recall": 0.0012903225806451613, "f1_score": 0.0025445292620865138}, {"class": "Vehicle Lot", "map@50:95": 0.014002680516415778, "map@50": 0.03791306589620266, "precision": 0.2336448598130841, "recall": 0.04091653027823241, "f1_score": 0.06963788300835655}, {"class": "Yacht", "map@50:95": 0.27353064724092324, "map@50": 0.6157179996623828, "precision": 0.49333333333333335, "recall": 0.7254901960784313, "f1_score": 0.5873015873015873}, {"class": "all", "map@50:95": 0.11501299697752468, "map@50": 0.23613258532485484, "precision": 0.32026371673288817, "recall": 0.2645686913827421, "f1_score": 0.257098145450584}], "map": 0.23613258532485484, "precision": 0.32026371673288817, "recall": 0.2645686913827421, "f1_score": 0.257098145450584}, "test_coco_eval_bbox": [0.11501299697752468, 0.23613258532485484, 0.09514516908755886, 0.08067823913752571, 0.14427286788113147, 0.19415069776351163, 0.0834498329332066, 0.214294987821286, 0.2705236663444596, 0.2055000333542548, 0.3195129589243432, 0.3708863726084828], "epoch": 5, "n_parameters": 126141694, "best_res": 0.11501299697752468, "best_ep": 5, "now_time": "2026-02-01 12:36:42.918926", "train_epoch_time": "1:11:14", "epoch_time": "1:17:58"}
{"train_lr": 9.999999999999097e-05, "train_class_error": 13.891100466467488, "train_loss": 10.213171951726357, "train_loss_ce": 0.7216019751397694, "train_loss_bbox": 0.15446072890035356, "train_loss_giou": 0.8073669242698961, "train_loss_ce_0": 0.7287356035808875, "train_loss_bbox_0": 0.16043910136219436, "train_loss_giou_0": 0.8262537735954903, "train_loss_ce_1": 0.7214344871472822, "train_loss_bbox_1": 0.15762827158726594, "train_loss_giou_1": 0.8193643009251829, "train_loss_ce_2": 0.7207528300015665, "train_loss_bbox_2": 0.15574639489296135, "train_loss_giou_2": 0.8126797955451681, "train_loss_ce_3": 0.7211067904823464, "train_loss_bbox_3": 0.15481717597613512, "train_loss_giou_3": 0.8093537310012727, "train_loss_ce_enc": 0.7181577950991285, "train_loss_bbox_enc": 0.16926551055703884, "train_loss_giou_enc": 0.8540067622689758, "train_loss_ce_unscaled": 0.7216019751397694, "train_class_error_unscaled": 13.891100466467488, "train_loss_bbox_unscaled": 0.030892145778075582, "train_loss_giou_unscaled": 0.40368346213494805, "train_cardinality_error_unscaled": 3831.76843123036, "train_loss_ce_0_unscaled": 0.7287356035808875, "train_loss_bbox_0_unscaled": 0.03208782028579132, "train_loss_giou_0_unscaled": 0.4131268867977452, "train_cardinality_error_0_unscaled": 3831.771573604061, "train_loss_ce_1_unscaled": 0.7214344871472822, "train_loss_bbox_1_unscaled": 0.031525654305986126, "train_loss_giou_1_unscaled": 0.40968215046259143, "train_cardinality_error_1_unscaled": 3831.5339618080734, "train_loss_ce_2_unscaled": 0.7207528300015665, "train_loss_bbox_2_unscaled": 0.031149278990503945, "train_loss_giou_2_unscaled": 0.40633989777258406, "train_cardinality_error_2_unscaled": 3831.0795262267343, "train_loss_ce_3_unscaled": 0.7211067904823464, "train_loss_bbox_3_unscaled": 0.03096343520454982, "train_loss_giou_3_unscaled": 0.40467686550063636, "train_cardinality_error_3_unscaled": 3831.4564902102975, "train_loss_ce_enc_unscaled": 0.7181577950991285, "train_loss_bbox_enc_unscaled": 0.03385310212914162, "train_loss_giou_enc_unscaled": 0.4270033811344879, "train_cardinality_error_enc_unscaled": 3831.769156393522, "test_class_error": 19.19507540990436, "test_loss": 10.642265988455879, "test_loss_ce": 0.8091263640873015, "test_loss_bbox": 0.1478480238774939, "test_loss_giou": 0.8011834078932565, "test_loss_ce_0": 0.8103695436507936, "test_loss_bbox_0": 0.15252890142301717, "test_loss_giou_0": 0.824235015028999, "test_loss_ce_1": 0.80453125, "test_loss_bbox_1": 0.15154074394631953, "test_loss_giou_1": 0.8191807365985144, "test_loss_ce_2": 0.8062400793650794, "test_loss_bbox_2": 0.14921056692503273, "test_loss_giou_2": 0.8081348447099564, "test_loss_ce_3": 0.8076512896825396, "test_loss_bbox_3": 0.14736533296664084, "test_loss_giou_3": 0.8028813703381826, "test_loss_ce_enc": 0.7900465029761905, "test_loss_bbox_enc": 0.16046532174661046, "test_loss_giou_enc": 0.849726710310058, "test_loss_ce_unscaled": 0.8091263640873015, "test_class_error_unscaled": 19.19507540990436, "test_loss_bbox_unscaled": 0.029569604722930798, "test_loss_giou_unscaled": 0.40059170394662824, "test_cardinality_error_unscaled": 236.7015873015873, "test_loss_ce_0_unscaled": 0.8103695436507936, "test_loss_bbox_0_unscaled": 0.030505780273871054, "test_loss_giou_0_unscaled": 0.4121175075144995, "test_cardinality_error_0_unscaled": 236.64444444444445, "test_loss_ce_1_unscaled": 0.80453125, "test_loss_bbox_1_unscaled": 0.030308148778442824, "test_loss_giou_1_unscaled": 0.4095903682992572, "test_cardinality_error_1_unscaled": 236.7225396825397, "test_loss_ce_2_unscaled": 0.8062400793650794, "test_loss_bbox_2_unscaled": 0.029842113421727268, "test_loss_giou_2_unscaled": 0.4040674223549782, "test_cardinality_error_2_unscaled": 236.57460317460317, "test_loss_ce_3_unscaled": 0.8076512896825396, "test_loss_bbox_3_unscaled": 0.029473066574908675, "test_loss_giou_3_unscaled": 0.4014406851690913, "test_cardinality_error_3_unscaled": 236.58666666666667, "test_loss_ce_enc_unscaled": 0.7900465029761905, "test_loss_bbox_enc_unscaled": 0.03209306437625653, "test_loss_giou_enc_unscaled": 0.424863355155029, "test_cardinality_error_enc_unscaled": 236.73333333333332, "test_results_json": {"class_map": [{"class": "Aircraft Hangar", "map@50:95": 0.018699239725430656, "map@50": 0.02208986231414977, "precision": 0.16666666666666666, "recall": 0.041666666666666664, "f1_score": 0.06666666666666667}, {"class": "Barge", "map@50:95": 0.1156227583688381, "map@50": 0.20860793384869328, "precision": 0.21978021978021978, "recall": 0.45454545454545453, "f1_score": 0.2962962962962963}, {"class": "Building", "map@50:95": 0.25727316778955084, "map@50": 0.5821588733741717, "precision": 0.5406144924379586, "recall": 0.671905930124355, "f1_score": 0.5991521076612051}, {"class": "Bus", "map@50:95": 0.10265103920070759, "map@50": 0.24419832787816673, "precision": 0.35908756438557765, "recall": 0.3448763250883392, "f1_score": 0.35183850036049025}, {"class": "Cargo Car", "map@50:95": 0.36891214484409385, "map@50": 0.6785885168587197, "precision": 0.38181818181818183, "recall": 0.7954545454545454, "f1_score": 0.5159705159705159}, {"class": "Cargo Plane", "map@50:95": 0.5030941402134522, "map@50": 0.7953353720762918, "precision": 0.6166666666666667, "recall": 0.8314606741573034, "f1_score": 0.7081339712918661}, {"class": "Cargo Truck", "map@50:95": 0.020580380759101343, "map@50": 0.0585361408277297, "precision": 0.1843137254901961, "recall": 0.12165660051768766, "f1_score": 0.14656964656964658}, {"class": "Cement Mixer", "map@50:95": 0.05906169544882979, "map@50": 0.14963502813710414, "precision": 1.0, "recall": 0.03636363636363636, "f1_score": 0.07017543859649122}, {"class": "Construction Site", "map@50:95": 0.0025645811555738136, "map@50": 0.007355873491390427, "precision": 0.022222222222222223, "recall": 0.007407407407407408, "f1_score": 0.011111111111111112}, {"class": "Container Crane", "map@50:95": 0.11770952079179495, "map@50": 0.18336464103854525, "precision": 0.11764705882352941, "recall": 0.21052631578947367, "f1_score": 0.1509433962264151}, {"class": "Container Ship", "map@50:95": 0.38902358607872023, "map@50": 0.5819765521337036, "precision": 0.6904761904761905, "recall": 0.5576923076923077, "f1_score": 0.6170212765957447}, {"class": "Crane Truck", "map@50:95": 0.015153195793833618, "map@50": 0.03971358039421281, "precision": 0.06666666666666667, "recall": 0.08695652173913043, "f1_score": 0.07547169811320756}, {"class": "Damaged Building", "map@50:95": 0.06500016788804983, "map@50": 0.1308207522582456, "precision": 0.25316455696202533, "recall": 0.21505376344086022, "f1_score": 0.23255813953488372}, {"class": "Dump Truck", "map@50:95": 0.05758585354145672, "map@50": 0.12372801146801961, "precision": 0.27564102564102566, "recall": 0.14429530201342283, "f1_score": 0.18942731277533043}, {"class": "Engineering Vehicle", "map@50:95": 0.016355001123420233, "map@50": 0.027236102080983395, "precision": 0.16666666666666666, "recall": 0.022222222222222223, "f1_score": 0.0392156862745098}, {"class": "Excavator", "map@50:95": 0.14540152351753555, "map@50": 0.3366739821030968, "precision": 0.2862190812720848, "recall": 0.44505494505494503, "f1_score": 0.3483870967741935}, {"class": "Facility", "map@50:95": 0.08584823842782958, "map@50": 0.180890354129485, "precision": 0.21487603305785125, "recall": 0.23853211009174313, "f1_score": 0.22608695652173916}, {"class": "Ferry", "map@50:95": 0.1877972405243051, "map@50": 0.2822556159745478, "precision": 0.5, "recall": 0.3333333333333333, "f1_score": 0.4}, {"class": "Fishing Vessel", "map@50:95": 0.054734415485860166, "map@50": 0.12552965260336194, "precision": 0.26785714285714285, "recall": 0.19480519480519481, "f1_score": 0.2255639097744361}, {"class": "Fixed-wing Aircraft", "map@50:95": 0.10047864554795247, "map@50": 0.261054831351861, "precision": 0.5, "recall": 0.3181818181818182, "f1_score": 0.3888888888888889}, {"class": "Flat Car", "map@50:95": 0.02984704910322975, "map@50": 0.13283984774532737, "precision": 0.3333333333333333, "recall": 0.020833333333333332, "f1_score": 0.0392156862745098}, {"class": "Front loader-Bulldozer", "map@50:95": 0.05928509200514812, "map@50": 0.15153612551514503, "precision": 0.21794871794871795, "recall": 0.24285714285714285, "f1_score": 0.22972972972972971}, {"class": "Ground Grader", "map@50:95": 0.0142153609644086, "map@50": 0.0385699096225412, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Haul Truck", "map@50:95": 0.5003513261882451, "map@50": 0.8871229238374855, "precision": 0.8925619834710744, "recall": 0.8571428571428571, "f1_score": 0.874493927125506}, {"class": "Helicopter", "map@50:95": 0.11772877217257835, "map@50": 0.36674381371531994, "precision": 0.17142857142857143, "recall": 0.5, "f1_score": 0.2553191489361702}, {"class": "Helipad", "map@50:95": 0.16478136691059345, "map@50": 0.30891606260898896, "precision": 0.5333333333333333, "recall": 0.3333333333333333, "f1_score": 0.4102564102564102}, {"class": "Hut-Tent", "map@50:95": 0.0019141514903835825, "map@50": 0.0043467155534773, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Locomotive", "map@50:95": 0.07157995094788144, "map@50": 0.14103511919483203, "precision": 0.15217391304347827, "recall": 0.3684210526315789, "f1_score": 0.21538461538461537}, {"class": "Maritime Vessel", "map@50:95": 0.13723404586008284, "map@50": 0.24047860100856058, "precision": 0.25384615384615383, "recall": 0.4125, "f1_score": 0.3142857142857143}, {"class": "Mobile Crane", "map@50:95": 0.03568808400422376, "map@50": 0.075932241136704, "precision": 0.23809523809523808, "recall": 0.09803921568627451, "f1_score": 0.1388888888888889}, {"class": "Motorboat", "map@50:95": 0.13200168205475, "map@50": 0.34942429517771106, "precision": 0.42346938775510207, "recall": 0.5, "f1_score": 0.4585635359116022}, {"class": "Oil Tanker", "map@50:95": 0.26367027554389655, "map@50": 0.34707774574925865, "precision": 0.11764705882352941, "recall": 0.3333333333333333, "f1_score": 0.1739130434782609}, {"class": "Passenger Car", "map@50:95": 0.3697681731609349, "map@50": 0.7434531998920371, "precision": 0.5703883495145631, "recall": 0.818815331010453, "f1_score": 0.6723891273247496}, {"class": "Passenger Vehicle", "map@50:95": 0.00025248420413126417, "map@50": 0.0007127423083132594, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Pickup Truck", "map@50:95": 0.0014353816933327427, "map@50": 0.004784960326852552, "precision": 0.028985507246376812, "recall": 0.011235955056179775, "f1_score": 0.016194331983805665}, {"class": "Pylon", "map@50:95": 0.27260002217700485, "map@50": 0.49401894291378134, "precision": 0.5862068965517241, "recall": 0.5, "f1_score": 0.5396825396825397}, {"class": "Reach Stacker", "map@50:95": 0.07686447216150184, "map@50": 0.11089323218036087, "precision": 0.3333333333333333, "recall": 0.1, "f1_score": 0.15384615384615383}, {"class": "Sailboat", "map@50:95": 0.23694522775616106, "map@50": 0.4742941831623328, "precision": 0.3504273504273504, "recall": 0.5616438356164384, "f1_score": 0.43157894736842106}, {"class": "Scraper-Tractor", "map@50:95": 0.004213789733777365, "map@50": 0.008368326483006488, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Shed", "map@50:95": 0.08363804334918656, "map@50": 0.18088114998843297, "precision": 0.3387096774193548, "recall": 0.21212121212121213, "f1_score": 0.2608695652173913}, {"class": "Shipping Container", "map@50:95": 0.0512565905110544, "map@50": 0.11794103778043973, "precision": 0.16795865633074936, "recall": 0.2838427947598253, "f1_score": 0.21103896103896103}, {"class": "Shipping container lot", "map@50:95": 0.13270731720987167, "map@50": 0.3032853569212269, "precision": 0.2895622895622896, "recall": 0.49710982658959535, "f1_score": 0.36595744680851067}, {"class": "Small Aircraft", "map@50:95": 0.32095093360972304, "map@50": 0.7205078717421728, "precision": 0.5952380952380952, "recall": 0.7936507936507936, "f1_score": 0.6802721088435374}, {"class": "Small Car", "map@50:95": 0.09720669394846218, "map@50": 0.3389314415101217, "precision": 0.4754752535609378, "recall": 0.43613532955440887, "f1_score": 0.4549564479563021}, {"class": "Storage Tank", "map@50:95": 0.29889293575347425, "map@50": 0.5404104317365463, "precision": 0.6585365853658537, "recall": 0.5242718446601942, "f1_score": 0.5837837837837838}, {"class": "Straddle Carrier", "map@50:95": 0.059696898890644244, "map@50": 0.1640149792197596, "precision": 0.21428571428571427, "recall": 0.42857142857142855, "f1_score": 0.2857142857142857}, {"class": "Tank car", "map@50:95": 0.3431264351470981, "map@50": 0.6998574924382072, "precision": 0.625, "recall": 0.6818181818181818, "f1_score": 0.6521739130434783}, {"class": "Tower", "map@50:95": 0.04701462778538652, "map@50": 0.11272162121818753, "precision": 0.375, "recall": 0.15789473684210525, "f1_score": 0.22222222222222218}, {"class": "Tower crane", "map@50:95": 0.007982870790499022, "map@50": 0.024632639148222033, "precision": 0.3333333333333333, "recall": 0.03571428571428571, "f1_score": 0.06451612903225806}, {"class": "Trailer", "map@50:95": 0.04339741669074365, "map@50": 0.108902266596418, "precision": 0.1794871794871795, "recall": 0.2980769230769231, "f1_score": 0.22405782137325764}, {"class": "Truck", "map@50:95": 0.034380357802527496, "map@50": 0.08953250891795651, "precision": 0.16146458583433373, "recall": 0.2258606213266163, "f1_score": 0.18830941547077354}, {"class": "Truck Tractor", "map@50:95": 0.004782421643274493, "map@50": 0.019858906024509675, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Truck w-Box", "map@50:95": 0.12188084253497684, "map@50": 0.3008570787770928, "precision": 0.3840104849279161, "recall": 0.38300653594771245, "f1_score": 0.3835078534031413}, {"class": "Truck w-Flatbed", "map@50:95": 0.02961108538455734, "map@50": 0.07748444940395526, "precision": 0.15976331360946747, "recall": 0.1836734693877551, "f1_score": 0.17088607594936708}, {"class": "Truck w-Liquid", "map@50:95": 0.0075685427517110685, "map@50": 0.020550452481145554, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tugboat", "map@50:95": 0.09231113027761612, "map@50": 0.2091191336341064, "precision": 0.2857142857142857, "recall": 0.1875, "f1_score": 0.22641509433962265}, {"class": "Utility Truck", "map@50:95": 0.010949628947337614, "map@50": 0.03193473038521212, "precision": 0.2073170731707317, "recall": 0.02193548387096774, "f1_score": 0.0396732788798133}, {"class": "Vehicle Lot", "map@50:95": 0.014843938627667388, "map@50": 0.04517944733073174, "precision": 0.18006430868167203, "recall": 0.09165302782324058, "f1_score": 0.12147505422993495}, {"class": "Yacht", "map@50:95": 0.39172434678625395, "map@50": 0.7457740446291897, "precision": 0.4339622641509434, "recall": 0.9019607843137255, "f1_score": 0.5859872611464968}, {"class": "all", "map@50:95": 0.12487875089492659, "map@50": 0.2500453564806466, "precision": 0.30675332865619676, "recall": 0.3063549787240634, "f1_score": 0.2763565616090483}], "map": 0.2500453564806466, "precision": 0.30675332865619676, "recall": 0.3063549787240634, "f1_score": 0.2763565616090483}, "test_coco_eval_bbox": [0.12487875089492659, 0.2500453564806466, 0.11131958574245633, 0.09049538137443582, 0.14278329641905577, 0.20595322477182978, 0.07745327079929505, 0.2248705512100995, 0.27922419593734793, 0.21694271104204993, 0.30753632872382797, 0.37546363639572755], "epoch": 6, "n_parameters": 126141694, "best_res": 0.12487875089492659, "best_ep": 6, "now_time": "2026-02-01 13:54:57.577351", "train_epoch_time": "1:11:26", "epoch_time": "1:18:14"}
{"train_lr": 9.999999999999097e-05, "train_class_error": 13.077874383518365, "train_loss": 10.005053484138875, "train_loss_ce": 0.7089565177777896, "train_loss_bbox": 0.149522309836382, "train_loss_giou": 0.7906020753115329, "train_loss_ce_0": 0.7164618686136862, "train_loss_bbox_0": 0.1546913090503374, "train_loss_giou_0": 0.8090187317034617, "train_loss_ce_1": 0.7083796660101448, "train_loss_bbox_1": 0.1523809079580587, "train_loss_giou_1": 0.8021578013461109, "train_loss_ce_2": 0.708241701735049, "train_loss_bbox_2": 0.15085050634171293, "train_loss_giou_2": 0.7962821878421694, "train_loss_ce_3": 0.7090490139267651, "train_loss_bbox_3": 0.14985341251577217, "train_loss_giou_3": 0.7920414104725438, "train_loss_ce_enc": 0.7069896169502708, "train_loss_bbox_enc": 0.1630673283488882, "train_loss_giou_enc": 0.8365071202099957, "train_loss_ce_unscaled": 0.7089565177777896, "train_class_error_unscaled": 13.077874383518365, "train_loss_bbox_unscaled": 0.0299044619771226, "train_loss_giou_unscaled": 0.39530103765576646, "train_cardinality_error_unscaled": 3831.8376843123037, "train_loss_ce_0_unscaled": 0.7164618686136862, "train_loss_bbox_0_unscaled": 0.03093826183283277, "train_loss_giou_0_unscaled": 0.40450936585173086, "train_cardinality_error_0_unscaled": 3832.1036983321246, "train_loss_ce_1_unscaled": 0.7083796660101448, "train_loss_bbox_1_unscaled": 0.030476181593989574, "train_loss_giou_1_unscaled": 0.40107890067305546, "train_cardinality_error_1_unscaled": 3832.0931834662797, "train_loss_ce_2_unscaled": 0.708241701735049, "train_loss_bbox_2_unscaled": 0.030170101287567856, "train_loss_giou_2_unscaled": 0.3981410939210847, "train_cardinality_error_2_unscaled": 3830.8436064781245, "train_loss_ce_3_unscaled": 0.7090490139267651, "train_loss_bbox_3_unscaled": 0.029970682495660744, "train_loss_giou_3_unscaled": 0.3960207052362719, "train_cardinality_error_3_unscaled": 3831.9184191443073, "train_loss_ce_enc_unscaled": 0.7069896169502708, "train_loss_bbox_enc_unscaled": 0.0326134656874327, "train_loss_giou_enc_unscaled": 0.41825356010499787, "train_cardinality_error_enc_unscaled": 3832.0818225767466, "test_class_error": 19.486074821835473, "test_loss": 10.73437296988472, "test_loss_ce": 0.8110664682539682, "test_loss_bbox": 0.15018344320770766, "test_loss_giou": 0.8127556103562552, "test_loss_ce_0": 0.8103475322420635, "test_loss_bbox_0": 0.15545642323673717, "test_loss_giou_0": 0.8336038968487391, "test_loss_ce_1": 0.805115947420635, "test_loss_bbox_1": 0.15375211656803175, "test_loss_giou_1": 0.8281276442134191, "test_loss_ce_2": 0.8086693948412699, "test_loss_bbox_2": 0.15197280991290296, "test_loss_giou_2": 0.8191418520704148, "test_loss_ce_3": 0.8101283482142857, "test_loss_bbox_3": 0.15070561952179387, "test_loss_giou_3": 0.8159807097628003, "test_loss_ce_enc": 0.7956355406746032, "test_loss_bbox_enc": 0.16305600254012953, "test_loss_giou_enc": 0.8586736212457929, "test_loss_ce_unscaled": 0.8110664682539682, "test_class_error_unscaled": 19.486074821835473, "test_loss_bbox_unscaled": 0.030036688689733783, "test_loss_giou_unscaled": 0.4063778051781276, "test_cardinality_error_unscaled": 236.6495238095238, "test_loss_ce_0_unscaled": 0.8103475322420635, "test_loss_bbox_0_unscaled": 0.03109128468344727, "test_loss_giou_0_unscaled": 0.4168019484243696, "test_cardinality_error_0_unscaled": 236.7174603174603, "test_loss_ce_1_unscaled": 0.805115947420635, "test_loss_bbox_1_unscaled": 0.0307504232950686, "test_loss_giou_1_unscaled": 0.41406382210670956, "test_cardinality_error_1_unscaled": 236.69015873015874, "test_loss_ce_2_unscaled": 0.8086693948412699, "test_loss_bbox_2_unscaled": 0.03039456193690144, "test_loss_giou_2_unscaled": 0.4095709260352074, "test_cardinality_error_2_unscaled": 236.5320634920635, "test_loss_ce_3_unscaled": 0.8101283482142857, "test_loss_bbox_3_unscaled": 0.030141123897647337, "test_loss_giou_3_unscaled": 0.4079903548814002, "test_cardinality_error_3_unscaled": 236.65968253968254, "test_loss_ce_enc_unscaled": 0.7956355406746032, "test_loss_bbox_enc_unscaled": 0.03261120049622915, "test_loss_giou_enc_unscaled": 0.42933681062289647, "test_cardinality_error_enc_unscaled": 236.68698412698413, "test_results_json": {"class_map": [{"class": "Aircraft Hangar", "map@50:95": 0.006876373197048128, "map@50": 0.010899021734406544, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Barge", "map@50:95": 0.10345750768861839, "map@50": 0.179637892431997, "precision": 0.23809523809523808, "recall": 0.11363636363636363, "f1_score": 0.15384615384615383}, {"class": "Building", "map@50:95": 0.25890258749567696, "map@50": 0.5828414552072346, "precision": 0.508140874705676, "recall": 0.6864393875306658, "f1_score": 0.5839840517024232}, {"class": "Bus", "map@50:95": 0.10103875795982467, "map@50": 0.2494896228213573, "precision": 0.36343115124153497, "recall": 0.3413427561837456, "f1_score": 0.3520408163265306}, {"class": "Cargo Car", "map@50:95": 0.34385440339274326, "map@50": 0.6418891147920882, "precision": 0.31722054380664655, "recall": 0.7954545454545454, "f1_score": 0.4535637149028078}, {"class": "Cargo Plane", "map@50:95": 0.48574225460696996, "map@50": 0.8478141840352205, "precision": 0.7575757575757576, "recall": 0.8426966292134831, "f1_score": 0.7978723404255319}, {"class": "Cargo Truck", "map@50:95": 0.017898436935130844, "map@50": 0.04620107197879926, "precision": 0.1531322505800464, "recall": 0.056945642795513375, "f1_score": 0.0830188679245283}, {"class": "Cement Mixer", "map@50:95": 0.08402456865919058, "map@50": 0.2371037530794482, "precision": 1.0, "recall": 0.07272727272727272, "f1_score": 0.13559322033898305}, {"class": "Construction Site", "map@50:95": 0.0036535536566213124, "map@50": 0.011266193843169429, "precision": 0.04819277108433735, "recall": 0.02962962962962963, "f1_score": 0.03669724770642202}, {"class": "Container Crane", "map@50:95": 0.09882200389307565, "map@50": 0.12822348333277336, "precision": 0.12280701754385964, "recall": 0.3684210526315789, "f1_score": 0.18421052631578946}, {"class": "Container Ship", "map@50:95": 0.4280400424401768, "map@50": 0.6138904840474364, "precision": 0.52, "recall": 0.75, "f1_score": 0.6141732283464567}, {"class": "Crane Truck", "map@50:95": 0.0364937392018023, "map@50": 0.07580352347778699, "precision": 0.25, "recall": 0.21739130434782608, "f1_score": 0.23255813953488372}, {"class": "Damaged Building", "map@50:95": 0.06252023595450654, "map@50": 0.12432141345050655, "precision": 0.19373219373219372, "recall": 0.24372759856630824, "f1_score": 0.21587301587301586}, {"class": "Dump Truck", "map@50:95": 0.0667342393895221, "map@50": 0.14587214155906655, "precision": 0.21348314606741572, "recall": 0.2550335570469799, "f1_score": 0.23241590214067276}, {"class": "Engineering Vehicle", "map@50:95": 0.026017159025081983, "map@50": 0.05264003975107294, "precision": 0.5, "recall": 0.044444444444444446, "f1_score": 0.0816326530612245}, {"class": "Excavator", "map@50:95": 0.1443025224074523, "map@50": 0.3012060480983794, "precision": 0.3087557603686636, "recall": 0.36813186813186816, "f1_score": 0.3358395989974937}, {"class": "Facility", "map@50:95": 0.11309408546970154, "map@50": 0.20939533201976218, "precision": 0.28205128205128205, "recall": 0.30275229357798167, "f1_score": 0.29203539823008845}, {"class": "Ferry", "map@50:95": 0.18902288095323277, "map@50": 0.2996872866574741, "precision": 0.21428571428571427, "recall": 0.5, "f1_score": 0.3}, {"class": "Fishing Vessel", "map@50:95": 0.0980718451046076, "map@50": 0.19923441830328786, "precision": 0.18518518518518517, "recall": 0.38961038961038963, "f1_score": 0.25104602510460244}, {"class": "Fixed-wing Aircraft", "map@50:95": 0.1280410944596686, "map@50": 0.38498368288461526, "precision": 0.5294117647058824, "recall": 0.4090909090909091, "f1_score": 0.46153846153846156}, {"class": "Flat Car", "map@50:95": 0.0637533308881159, "map@50": 0.1783292787987307, "precision": 0.6666666666666666, "recall": 0.041666666666666664, "f1_score": 0.0784313725490196}, {"class": "Front loader-Bulldozer", "map@50:95": 0.05205937395112764, "map@50": 0.15121857771839053, "precision": 0.2484076433121019, "recall": 0.2785714285714286, "f1_score": 0.2626262626262626}, {"class": "Ground Grader", "map@50:95": 0.011436944898952774, "map@50": 0.026036756166222818, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Haul Truck", "map@50:95": 0.5135123551172984, "map@50": 0.9234022270976434, "precision": 0.8721804511278195, "recall": 0.9206349206349206, "f1_score": 0.8957528957528956}, {"class": "Helicopter", "map@50:95": 0.09144475161801893, "map@50": 0.3821857185718571, "precision": 0.5555555555555556, "recall": 0.4166666666666667, "f1_score": 0.4761904761904762}, {"class": "Helipad", "map@50:95": 0.1440447263183397, "map@50": 0.33104407744272496, "precision": 0.5238095238095238, "recall": 0.4583333333333333, "f1_score": 0.4888888888888889}, {"class": "Hut-Tent", "map@50:95": 0.0005732593916936351, "map@50": 0.0017000432801009525, "precision": 0.03571428571428571, "recall": 0.012658227848101266, "f1_score": 0.01869158878504673}, {"class": "Locomotive", "map@50:95": 0.050800496514178214, "map@50": 0.10615016932489518, "precision": 0.18181818181818182, "recall": 0.10526315789473684, "f1_score": 0.13333333333333333}, {"class": "Maritime Vessel", "map@50:95": 0.11425819878565549, "map@50": 0.2028900975826172, "precision": 0.16908212560386474, "recall": 0.4375, "f1_score": 0.24390243902439024}, {"class": "Mobile Crane", "map@50:95": 0.025020455652182298, "map@50": 0.04936006897119452, "precision": 0.23076923076923078, "recall": 0.11764705882352941, "f1_score": 0.15584415584415587}, {"class": "Motorboat", "map@50:95": 0.1256129178542795, "map@50": 0.37561933733262026, "precision": 0.32971014492753625, "recall": 0.5481927710843374, "f1_score": 0.411764705882353}, {"class": "Oil Tanker", "map@50:95": 0.2542142931429075, "map@50": 0.34328755456190796, "precision": 0.3333333333333333, "recall": 0.3333333333333333, "f1_score": 0.3333333333333333}, {"class": "Passenger Car", "map@50:95": 0.37829659312736663, "map@50": 0.7583732429046303, "precision": 0.6391184573002755, "recall": 0.8083623693379791, "f1_score": 0.7138461538461539}, {"class": "Passenger Vehicle", "map@50:95": 0.002250569910584134, "map@50": 0.006436476139328648, "precision": 0.06451612903225806, "recall": 0.007518796992481203, "f1_score": 0.013468013468013467}, {"class": "Pickup Truck", "map@50:95": 0.004300865148877015, "map@50": 0.013913210726943443, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Pylon", "map@50:95": 0.2977615312668533, "map@50": 0.5304104651030557, "precision": 0.5873015873015873, "recall": 0.5441176470588235, "f1_score": 0.564885496183206}, {"class": "Reach Stacker", "map@50:95": 0.07110676534490648, "map@50": 0.16606545544011736, "precision": 0.5, "recall": 0.2, "f1_score": 0.28571428571428575}, {"class": "Sailboat", "map@50:95": 0.22561179599691306, "map@50": 0.45141749380154367, "precision": 0.43902439024390244, "recall": 0.4931506849315068, "f1_score": 0.4645161290322581}, {"class": "Scraper-Tractor", "map@50:95": 0.008266199290736527, "map@50": 0.01869093741672304, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Shed", "map@50:95": 0.07602531281052932, "map@50": 0.15721924913420623, "precision": 0.40145985401459855, "recall": 0.18518518518518517, "f1_score": 0.2534562211981567}, {"class": "Shipping Container", "map@50:95": 0.04976696773233576, "map@50": 0.11239627229689644, "precision": 0.20642201834862386, "recall": 0.1965065502183406, "f1_score": 0.20134228187919462}, {"class": "Shipping container lot", "map@50:95": 0.12432866053773191, "map@50": 0.2642937555032208, "precision": 0.18331716779825413, "recall": 0.546242774566474, "f1_score": 0.27450980392156865}, {"class": "Small Aircraft", "map@50:95": 0.3148952142376157, "map@50": 0.6583846033944025, "precision": 0.5384615384615384, "recall": 0.7777777777777778, "f1_score": 0.6363636363636364}, {"class": "Small Car", "map@50:95": 0.09170959187743752, "map@50": 0.33058653852227654, "precision": 0.49438867667525394, "recall": 0.4243918558247121, "f1_score": 0.4567239304081409}, {"class": "Storage Tank", "map@50:95": 0.2983048398074821, "map@50": 0.5529130050374809, "precision": 0.47869674185463656, "recall": 0.6181229773462783, "f1_score": 0.53954802259887}, {"class": "Straddle Carrier", "map@50:95": 0.07962499922028922, "map@50": 0.15755781879816214, "precision": 0.16666666666666666, "recall": 0.5714285714285714, "f1_score": 0.25806451612903225}, {"class": "Tank car", "map@50:95": 0.3635012551421014, "map@50": 0.7181380044900824, "precision": 0.6296296296296297, "recall": 0.7727272727272727, "f1_score": 0.6938775510204083}, {"class": "Tower", "map@50:95": 0.026079998408428097, "map@50": 0.09625217196234856, "precision": 0.3333333333333333, "recall": 0.21052631578947367, "f1_score": 0.2580645161290323}, {"class": "Tower crane", "map@50:95": 0.027031337498887177, "map@50": 0.050507382279685493, "precision": 0.16666666666666666, "recall": 0.07142857142857142, "f1_score": 0.1}, {"class": "Trailer", "map@50:95": 0.04052552734346508, "map@50": 0.10082397985484323, "precision": 0.32456140350877194, "recall": 0.10164835164835165, "f1_score": 0.15481171548117156}, {"class": "Truck", "map@50:95": 0.033748072128318415, "map@50": 0.09664336038422591, "precision": 0.22290161200667039, "recall": 0.16834592779177163, "f1_score": 0.1918201387227936}, {"class": "Truck Tractor", "map@50:95": 0.001812427922301176, "map@50": 0.007466198232726498, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Truck w-Box", "map@50:95": 0.11616644024298939, "map@50": 0.27585333664178585, "precision": 0.42492492492492495, "recall": 0.36993464052287583, "f1_score": 0.39552760307477286}, {"class": "Truck w-Flatbed", "map@50:95": 0.026422858307832104, "map@50": 0.07231469846590795, "precision": 0.34210526315789475, "recall": 0.08843537414965986, "f1_score": 0.14054054054054055}, {"class": "Truck w-Liquid", "map@50:95": 0.0461669997265846, "map@50": 0.1044441581719406, "precision": 0.6, "recall": 0.07894736842105263, "f1_score": 0.1395348837209302}, {"class": "Tugboat", "map@50:95": 0.08628445690737474, "map@50": 0.19063558260756616, "precision": 0.25, "recall": 0.34375, "f1_score": 0.2894736842105263}, {"class": "Utility Truck", "map@50:95": 0.008955769222509118, "map@50": 0.03445502615864314, "precision": 0.12266666666666666, "recall": 0.05935483870967742, "f1_score": 0.07999999999999999}, {"class": "Vehicle Lot", "map@50:95": 0.014575129745549916, "map@50": 0.036704351254123906, "precision": 0.20388349514563106, "recall": 0.03436988543371522, "f1_score": 0.058823529411764705}, {"class": "Yacht", "map@50:95": 0.2751037848176338, "map@50": 0.5463358385063424, "precision": 0.3357664233576642, "recall": 0.9019607843137255, "f1_score": 0.48936170212765956}, {"class": "all", "map@50:95": 0.12427056541944126, "map@50": 0.25292977429803387, "precision": 0.3306501769451353, "recall": 0.32257928357764126, "f1_score": 0.2873046299950566}], "map": 0.25292977429803387, "precision": 0.3306501769451353, "recall": 0.32257928357764126, "f1_score": 0.2873046299950566}, "test_coco_eval_bbox": [0.12427056541944126, 0.25292977429803387, 0.10615689846605555, 0.09205544320448122, 0.1480726422617448, 0.20475817721634812, 0.08173008015587875, 0.22652582122581613, 0.28262782060194963, 0.2147947371323488, 0.3283791321178229, 0.382089991354912], "epoch": 7, "n_parameters": 126141694, "best_res": 0.12487875089492659, "best_ep": 6, "now_time": "2026-02-01 15:12:59.660562", "train_epoch_time": "1:11:23", "epoch_time": "1:18:01"}
{"train_lr": 9.999999999999097e-05, "train_class_error": 12.42035167943979, "train_loss": 9.871964096449263, "train_loss_ce": 0.7041397857401691, "train_loss_bbox": 0.14504520802972273, "train_loss_giou": 0.7757681587182844, "train_loss_ce_0": 0.7150387441714846, "train_loss_bbox_0": 0.15053005485085408, "train_loss_giou_0": 0.7949739877816225, "train_loss_ce_1": 0.7054262256208363, "train_loss_bbox_1": 0.14794049859241665, "train_loss_giou_1": 0.787676557182639, "train_loss_ce_2": 0.7043165310130878, "train_loss_bbox_2": 0.1464248736926906, "train_loss_giou_2": 0.7814296468595303, "train_loss_ce_3": 0.7041712051349734, "train_loss_bbox_3": 0.14547762688873384, "train_loss_giou_3": 0.7774067964032922, "train_loss_ce_enc": 0.7041783156449726, "train_loss_bbox_enc": 0.15931661247982873, "train_loss_giou_enc": 0.8227032618365276, "train_loss_ce_unscaled": 0.7041397857401691, "train_class_error_unscaled": 12.42035167943979, "train_loss_bbox_unscaled": 0.029009041622355815, "train_loss_giou_unscaled": 0.3878840793591422, "train_cardinality_error_unscaled": 3834.4619289340103, "train_loss_ce_0_unscaled": 0.7150387441714846, "train_loss_bbox_0_unscaled": 0.030106010998341808, "train_loss_giou_0_unscaled": 0.39748699389081127, "train_cardinality_error_0_unscaled": 3834.705704616872, "train_loss_ce_1_unscaled": 0.7054262256208363, "train_loss_bbox_1_unscaled": 0.029588099730971876, "train_loss_giou_1_unscaled": 0.3938382785913195, "train_cardinality_error_1_unscaled": 3834.0557167029247, "train_loss_ce_2_unscaled": 0.7043165310130878, "train_loss_bbox_2_unscaled": 0.02928497475435001, "train_loss_giou_2_unscaled": 0.39071482342976516, "train_cardinality_error_2_unscaled": 3833.8747884940776, "train_loss_ce_3_unscaled": 0.7041712051349734, "train_loss_bbox_3_unscaled": 0.029095525385634417, "train_loss_giou_3_unscaled": 0.3887033982016461, "train_cardinality_error_3_unscaled": 3834.5514865844816, "train_loss_ce_enc_unscaled": 0.7041783156449726, "train_loss_bbox_enc_unscaled": 0.031863322490377134, "train_loss_giou_enc_unscaled": 0.4113516309182638, "train_cardinality_error_enc_unscaled": 3834.6246072032873, "test_class_error": 18.72856178404793, "test_loss": 10.677576532061138, "test_loss_ce": 0.8067162698412699, "test_loss_bbox": 0.14685972370739495, "test_loss_giou": 0.8129892556251042, "test_loss_ce_0": 0.8031591021825397, "test_loss_bbox_0": 0.15315370539055456, "test_loss_giou_0": 0.832567593691841, "test_loss_ce_1": 0.8018303571428571, "test_loss_bbox_1": 0.14916604239020556, "test_loss_giou_1": 0.822898630698522, "test_loss_ce_2": 0.8049274553571428, "test_loss_bbox_2": 0.14713245261164884, "test_loss_giou_2": 0.815093373864416, "test_loss_ce_3": 0.8060069444444444, "test_loss_bbox_3": 0.14725542897506366, "test_loss_giou_3": 0.8147082136263923, "test_loss_ce_enc": 0.7904743303571429, "test_loss_bbox_enc": 0.16069011886796308, "test_loss_giou_enc": 0.8619475353142572, "test_loss_ce_unscaled": 0.8067162698412699, "test_class_error_unscaled": 18.72856178404793, "test_loss_bbox_unscaled": 0.029371944700944282, "test_loss_giou_unscaled": 0.4064946278125521, "test_cardinality_error_unscaled": 236.74412698412698, "test_loss_ce_0_unscaled": 0.8031591021825397, "test_loss_bbox_0_unscaled": 0.030630741032756984, "test_loss_giou_0_unscaled": 0.4162837968459205, "test_cardinality_error_0_unscaled": 236.7384126984127, "test_loss_ce_1_unscaled": 0.8018303571428571, "test_loss_bbox_1_unscaled": 0.029833208484841243, "test_loss_giou_1_unscaled": 0.411449315349261, "test_cardinality_error_1_unscaled": 236.75174603174602, "test_loss_ce_2_unscaled": 0.8049274553571428, "test_loss_bbox_2_unscaled": 0.029426490535131758, "test_loss_giou_2_unscaled": 0.407546686932208, "test_cardinality_error_2_unscaled": 236.7352380952381, "test_loss_ce_3_unscaled": 0.8060069444444444, "test_loss_bbox_3_unscaled": 0.02945108580683905, "test_loss_giou_3_unscaled": 0.40735410681319617, "test_cardinality_error_3_unscaled": 236.72761904761904, "test_loss_ce_enc_unscaled": 0.7904743303571429, "test_loss_bbox_enc_unscaled": 0.032138023752837426, "test_loss_giou_enc_unscaled": 0.4309737676571286, "test_cardinality_error_enc_unscaled": 236.75174603174602, "test_results_json": {"class_map": [{"class": "Aircraft Hangar", "map@50:95": 0.022426733686735315, "map@50": 0.03400713098569541, "precision": 0.09090909090909091, "recall": 0.041666666666666664, "f1_score": 0.05714285714285715}, {"class": "Barge", "map@50:95": 0.1478860205537576, "map@50": 0.2776919387990519, "precision": 0.44, "recall": 0.25, "f1_score": 0.31884057971014496}, {"class": "Building", "map@50:95": 0.2526163743415668, "map@50": 0.5791012694181455, "precision": 0.5723585378067101, "recall": 0.6575078250571018, "f1_score": 0.6119855436485753}, {"class": "Bus", "map@50:95": 0.09790411501042513, "map@50": 0.23835926725633133, "precision": 0.32665832290362956, "recall": 0.36890459363957595, "f1_score": 0.34649850647195485}, {"class": "Cargo Car", "map@50:95": 0.33799473178556494, "map@50": 0.6852908123657403, "precision": 0.4484304932735426, "recall": 0.7575757575757576, "f1_score": 0.5633802816901409}, {"class": "Cargo Plane", "map@50:95": 0.504297641856429, "map@50": 0.8496170920305723, "precision": 0.7628865979381443, "recall": 0.8314606741573034, "f1_score": 0.7956989247311828}, {"class": "Cargo Truck", "map@50:95": 0.02212115181787381, "map@50": 0.06835889419459841, "precision": 0.13528413910093298, "recall": 0.27523727351164795, "f1_score": 0.1814046061984646}, {"class": "Cement Mixer", "map@50:95": 0.06954497204681086, "map@50": 0.14262020392261648, "precision": 0.37037037037037035, "recall": 0.18181818181818182, "f1_score": 0.24390243902439027}, {"class": "Construction Site", "map@50:95": 0.0019477393354032854, "map@50": 0.008605783204823164, "precision": 0.05263157894736842, "recall": 0.022222222222222223, "f1_score": 0.03125}, {"class": "Container Crane", "map@50:95": 0.07217157368986053, "map@50": 0.11984448908138738, "precision": 0.11428571428571428, "recall": 0.21052631578947367, "f1_score": 0.14814814814814814}, {"class": "Container Ship", "map@50:95": 0.3929848393472798, "map@50": 0.5658515768924237, "precision": 0.5692307692307692, "recall": 0.7115384615384616, "f1_score": 0.6324786324786325}, {"class": "Crane Truck", "map@50:95": 0.05686834652289322, "map@50": 0.10975835125040741, "precision": 0.125, "recall": 0.13043478260869565, "f1_score": 0.1276595744680851}, {"class": "Damaged Building", "map@50:95": 0.05602512915686647, "map@50": 0.1129585703417978, "precision": 0.23048327137546468, "recall": 0.2222222222222222, "f1_score": 0.22627737226277372}, {"class": "Dump Truck", "map@50:95": 0.061069210794609065, "map@50": 0.14048581053661488, "precision": 0.24154589371980675, "recall": 0.16778523489932887, "f1_score": 0.19801980198019803}, {"class": "Engineering Vehicle", "map@50:95": 0.04871267944842791, "map@50": 0.09660735426689043, "precision": 0.2777777777777778, "recall": 0.1111111111111111, "f1_score": 0.15873015873015872}, {"class": "Excavator", "map@50:95": 0.14244928799092202, "map@50": 0.3215710606677696, "precision": 0.45588235294117646, "recall": 0.34065934065934067, "f1_score": 0.38993710691823896}, {"class": "Facility", "map@50:95": 0.08466407842825564, "map@50": 0.1623253391913378, "precision": 0.12608695652173912, "recall": 0.26605504587155965, "f1_score": 0.1710914454277286}, {"class": "Ferry", "map@50:95": 0.1882556984729776, "map@50": 0.2900980772202993, "precision": 0.3333333333333333, "recall": 0.2916666666666667, "f1_score": 0.3111111111111111}, {"class": "Fishing Vessel", "map@50:95": 0.0596123883278337, "map@50": 0.12846174393787804, "precision": 0.17647058823529413, "recall": 0.11688311688311688, "f1_score": 0.140625}, {"class": "Fixed-wing Aircraft", "map@50:95": 0.14460235420824571, "map@50": 0.34637279819936007, "precision": 0.38461538461538464, "recall": 0.22727272727272727, "f1_score": 0.2857142857142857}, {"class": "Flat Car", "map@50:95": 0.04175655985469949, "map@50": 0.08778109019191299, "precision": 1.0, "recall": 0.020833333333333332, "f1_score": 0.04081632653061225}, {"class": "Front loader-Bulldozer", "map@50:95": 0.05581100878625894, "map@50": 0.15340895184313674, "precision": 0.32894736842105265, "recall": 0.17857142857142858, "f1_score": 0.23148148148148148}, {"class": "Ground Grader", "map@50:95": 0.020897796255340897, "map@50": 0.04323610532788857, "precision": 0.05555555555555555, "recall": 0.13333333333333333, "f1_score": 0.0784313725490196}, {"class": "Haul Truck", "map@50:95": 0.5199724642465389, "map@50": 0.9350906441303098, "precision": 0.8712121212121212, "recall": 0.9126984126984127, "f1_score": 0.8914728682170543}, {"class": "Helicopter", "map@50:95": 0.1593116454502593, "map@50": 0.4276284771334276, "precision": 0.75, "recall": 0.25, "f1_score": 0.375}, {"class": "Helipad", "map@50:95": 0.19337419185034488, "map@50": 0.31563956962687517, "precision": 0.47058823529411764, "recall": 0.3333333333333333, "f1_score": 0.39024390243902435}, {"class": "Hut-Tent", "map@50:95": 0.00021243282656364525, "map@50": 0.00047883851377110285, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Locomotive", "map@50:95": 0.04929023332620413, "map@50": 0.08717009436792734, "precision": 0.0975609756097561, "recall": 0.21052631578947367, "f1_score": 0.13333333333333333}, {"class": "Maritime Vessel", "map@50:95": 0.11886080181970426, "map@50": 0.21508673309411752, "precision": 0.2111111111111111, "recall": 0.475, "f1_score": 0.29230769230769227}, {"class": "Mobile Crane", "map@50:95": 0.047169682976732866, "map@50": 0.10350276891455235, "precision": 0.20454545454545456, "recall": 0.17647058823529413, "f1_score": 0.18947368421052632}, {"class": "Motorboat", "map@50:95": 0.12963546432787806, "map@50": 0.3969859830461762, "precision": 0.3798449612403101, "recall": 0.5903614457831325, "f1_score": 0.46226415094339623}, {"class": "Oil Tanker", "map@50:95": 0.15264232673267322, "map@50": 0.2216171617161716, "precision": 0.18181818181818182, "recall": 0.3333333333333333, "f1_score": 0.23529411764705885}, {"class": "Passenger Car", "map@50:95": 0.3440328268632847, "map@50": 0.7374361744714963, "precision": 0.5272331154684096, "recall": 0.8432055749128919, "f1_score": 0.6487935656836461}, {"class": "Passenger Vehicle", "map@50:95": 0.0034768827448961705, "map@50": 0.011784498612336393, "precision": 0.05555555555555555, "recall": 0.0018796992481203006, "f1_score": 0.0036363636363636364}, {"class": "Pickup Truck", "map@50:95": 0.0020685681203422446, "map@50": 0.007235169573296642, "precision": 0.05660377358490566, "recall": 0.016853932584269662, "f1_score": 0.025974025974025972}, {"class": "Pylon", "map@50:95": 0.22793122077532219, "map@50": 0.4093437543854479, "precision": 0.6363636363636364, "recall": 0.4117647058823529, "f1_score": 0.5}, {"class": "Reach Stacker", "map@50:95": 0.03795379537953795, "map@50": 0.0759075907590759, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Sailboat", "map@50:95": 0.20390261780851474, "map@50": 0.4894090142310709, "precision": 0.27011494252873564, "recall": 0.6438356164383562, "f1_score": 0.38056680161943324}, {"class": "Scraper-Tractor", "map@50:95": 0.02885659533695305, "map@50": 0.18811881188118812, "precision": 1.0, "recall": 0.18181818181818182, "f1_score": 0.3076923076923077}, {"class": "Shed", "map@50:95": 0.08461012694632926, "map@50": 0.16805503068130218, "precision": 0.494949494949495, "recall": 0.16498316498316498, "f1_score": 0.2474747474747475}, {"class": "Shipping Container", "map@50:95": 0.036748563400303504, "map@50": 0.08652180139413984, "precision": 0.1984126984126984, "recall": 0.1091703056768559, "f1_score": 0.14084507042253522}, {"class": "Shipping container lot", "map@50:95": 0.1291319040923405, "map@50": 0.27378161620806357, "precision": 0.20958083832335328, "recall": 0.5057803468208093, "f1_score": 0.29635901778154106}, {"class": "Small Aircraft", "map@50:95": 0.31417554691024896, "map@50": 0.7221541574951577, "precision": 0.51, "recall": 0.8095238095238095, "f1_score": 0.6257668711656442}, {"class": "Small Car", "map@50:95": 0.0813329932434381, "map@50": 0.31081939821487264, "precision": 0.39954243880118967, "recall": 0.44391347449226, "f1_score": 0.42056086019433847}, {"class": "Storage Tank", "map@50:95": 0.30341548547824504, "map@50": 0.5339307953363476, "precision": 0.559375, "recall": 0.5792880258899676, "f1_score": 0.5691573926868044}, {"class": "Straddle Carrier", "map@50:95": 0.10182557358269231, "map@50": 0.2536350493437352, "precision": 0.3333333333333333, "recall": 0.14285714285714285, "f1_score": 0.2}, {"class": "Tank car", "map@50:95": 0.22680118011801179, "map@50": 0.6678494772554178, "precision": 0.6666666666666666, "recall": 0.7272727272727273, "f1_score": 0.6956521739130435}, {"class": "Tower", "map@50:95": 0.042485696342913645, "map@50": 0.07811760933178338, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tower crane", "map@50:95": 0.015557602271855094, "map@50": 0.02564608620330472, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Trailer", "map@50:95": 0.04088080518195417, "map@50": 0.10719858181994169, "precision": 0.2072072072072072, "recall": 0.25274725274725274, "f1_score": 0.22772277227722773}, {"class": "Truck", "map@50:95": 0.03272542671143354, "map@50": 0.09423923830853541, "precision": 0.17598557072919352, "recall": 0.2867338371116709, "f1_score": 0.21810633881526428}, {"class": "Truck Tractor", "map@50:95": 0.0013074102921293558, "map@50": 0.004965601811666552, "precision": 0.04, "recall": 0.011976047904191617, "f1_score": 0.018433179723502304}, {"class": "Truck w-Box", "map@50:95": 0.12050386544038998, "map@50": 0.27899033679629287, "precision": 0.3966189856957087, "recall": 0.39869281045751637, "f1_score": 0.39765319426336376}, {"class": "Truck w-Flatbed", "map@50:95": 0.020109888694987345, "map@50": 0.06175249481848181, "precision": 0.1348314606741573, "recall": 0.16326530612244897, "f1_score": 0.1476923076923077}, {"class": "Truck w-Liquid", "map@50:95": 0.003978417857484336, "map@50": 0.012172233706887173, "precision": 0.0, "recall": 0.0, "f1_score": 0.0}, {"class": "Tugboat", "map@50:95": 0.11101541763370501, "map@50": 0.22393129131888112, "precision": 0.16822429906542055, "recall": 0.5625, "f1_score": 0.2589928057553957}, {"class": "Utility Truck", "map@50:95": 0.013591993637222216, "map@50": 0.038204754682865445, "precision": 0.14655172413793102, "recall": 0.04387096774193548, "f1_score": 0.06752730883813307}, {"class": "Vehicle Lot", "map@50:95": 0.015258196208361366, "map@50": 0.04031907385367081, "precision": 0.18050541516245489, "recall": 0.08183306055646482, "f1_score": 0.11261261261261261}, {"class": "Yacht", "map@50:95": 0.30839427763862126, "map@50": 0.6309483080934954, "precision": 0.4375, "recall": 0.6862745098039216, "f1_score": 0.5343511450381679}, {"class": "all", "map@50:95": 0.12042648396588908, "map@50": 0.2508149480043858, "precision": 0.3150945982161688, "recall": 0.30283136009200934, "f1_score": 0.2763658672674013}], "map": 0.2508149480043858, "precision": 0.3150945982161688, "recall": 0.30283136009200934, "f1_score": 0.2763658672674013}, "test_coco_eval_bbox": [0.12042648396588908, 0.2508149480043858, 0.09681127236558246, 0.08874319772947314, 0.151286919811246, 0.19891355540198172, 0.07802604082244642, 0.21198365965627858, 0.26988472159051863, 0.20398133020254328, 0.3191586622119291, 0.34707673629679064], "epoch": 8, "n_parameters": 126141694, "best_res": 0.12487875089492659, "best_ep": 6, "now_time": "2026-02-01 16:30:59.887901", "train_epoch_time": "1:11:27", "epoch_time": "1:18:00"}
